# OASIS Alzheimer Classification - Kaggle Notebook (2x T4 GPU)

This notebook runs the complete NIC project pipeline directly on Kaggle.

**Prerequisites:**
- Add the dataset `ninadaithal/imagesoasis` to your Kaggle notebook
- Enable GPU accelerator (2x T4 recommended)

**Pipeline Stages:**
1. Install dependencies
2. GPU/CUDA verification  
3. Imports & configuration
4. Data loading (from `/kaggle/input/imagesoasis`)
5. ACO Feature Selection
6. Phase 1: Hyperparameter Optimization (DE, GWO, PSO, BAT)
7. Phase 2: SA-tuned WOA and GWO
8. XAI Optimization
9. Final Training & Explanations

In [1]:
# Cell 1: Install missing dependencies
!pip install -q timm shap

In [2]:
# Cell 2: GPU/CUDA Check
import torch
import subprocess
import platform

print('=' * 50)
print('SYSTEM INFO')
print('=' * 50)
print(f'Python: {platform.python_version()}')
print(f'PyTorch: {torch.__version__}')
print(f'CUDA available: {torch.cuda.is_available()}')
print(f'GPU count: {torch.cuda.device_count()}')

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f'GPU {i}: {torch.cuda.get_device_name(i)}')
    print(f'Current GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB')
    
print('=' * 50)
!nvidia-smi

SYSTEM INFO
Python: 3.12.12
PyTorch: 2.8.0+cu126
CUDA available: True
GPU count: 2
GPU 0: Tesla T4
GPU 1: Tesla T4
Current GPU memory: 15.83 GB
Sat Dec 20 21:05:49 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8             11W /   70W |       3MiB /  15360MiB |      0%      Defaul

In [3]:
# Cell 3: Imports & Global Setup
import os
import sys
import json
import time
import random
import hashlib
import warnings
import traceback
import gc
import shutil
from datetime import datetime
from dataclasses import dataclass, field, asdict
from typing import Dict, List, Tuple, Optional, Callable, Any, Union
from pathlib import Path
from collections import Counter
from copy import deepcopy

import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')  # Non-interactive backend for Kaggle
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset, WeightedRandomSampler
from torch.cuda.amp import autocast, GradScaler
import torchvision.transforms as T

# Attempt to import timm for EfficientNet
try:
    import timm
    TIMM_AVAILABLE = True
except ImportError:
    TIMM_AVAILABLE = False
    print("Warning: timm not available. Will use torchvision EfficientNet.")

# Attempt to import torchvision models as fallback
from torchvision import models as tv_models

# sklearn for metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score, f1_score,
    precision_score, recall_score, confusion_matrix, classification_report
)
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# SHAP for explanations
try:
    import shap
    SHAP_AVAILABLE = True
except ImportError:
    SHAP_AVAILABLE = False
    print("Warning: SHAP not available. SHAP explanations will be skipped.")

# CV2 for image processing (optional, use PIL as fallback)
try:
    import cv2
    CV2_AVAILABLE = True
except ImportError:
    CV2_AVAILABLE = False

# Suppress warnings
warnings.filterwarnings('ignore')

# Version info
SCRIPT_VERSION = "1.0.0"
print(f"Script version: {SCRIPT_VERSION}")
print(f"PyTorch version: {torch.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"TIMM available: {TIMM_AVAILABLE}")
print(f"SHAP available: {SHAP_AVAILABLE}")

def set_global_seed(seed: int):
    """Set all random seeds for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

def get_device(preferred: str = "auto") -> torch.device:
    """Get computation device."""
    if preferred == "auto":
        if torch.cuda.is_available():
            return torch.device("cuda")
        else:
            return torch.device("cpu")
    return torch.device(preferred)

# Global constants
CLASS_NAMES = ["Mild Demented", "Moderate Demented", "Non Demented", "Very Mild Demented"]
NUM_CLASSES = 4

print(f"\nDevice: {get_device()}")

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Script version: 1.0.0
PyTorch version: 2.8.0+cu126
NumPy version: 2.0.2
TIMM available: True
SHAP available: True

Device: cuda


In [4]:
# Cell 4: Configuration

@dataclass
class Config:
    """Centralized configuration for all experiments."""
    # Mode
    TEST_RUN: bool = False  # Set to True for quick test, False for full run
    
    # Paths - KAGGLE SPECIFIC
    DATA_DIR: str = "/kaggle/input/imagesoasis"  # Kaggle dataset path
    OUTPUT_DIR: str = "/kaggle/working/runs"
    
    # Model
    IMG_SIZE: int = 224
    BACKBONE: str = "efficientnet_b0"
    PRETRAINED: bool = True
    DROPOUT: float = 0.3
    
    # Training
    BATCH_SIZE: int = 64  # Larger batch = faster training on T4
    EPOCHS: int = 2  # Reduced for speed
    LR: float = 1e-3
    WEIGHT_DECAY: float = 1e-4
    OPTIMIZER: str = "AdamW"
    USE_AMP: bool = True
    NUM_WORKERS: int = 4  # Keep at 4 for Kaggle
    
    # Early stopping
    EARLY_STOPPING_PATIENCE: int = 2  # Fast early stopping
    
    # Imbalance handling
    USE_WEIGHTED_SAMPLER: bool = True
    USE_CLASS_WEIGHTS: bool = True
    USE_FOCAL_LOSS: bool = False
    FOCAL_LOSS_GAMMA: float = 2.0
    
    # Augmentation
    AUGMENT_STRENGTH: float = 1.0
    
    # Freeze/unfreeze
    FREEZE_BACKBONE_EPOCHS: int = 2
    
    # Metaheuristic budgets - OPTIMIZED FOR SPEED
    META_POPULATION: int = 4  # Minimum viable (required for DE)
    META_ITERATIONS: int = 2  # Just 2 iterations
    META_EVAL_EPOCHS: int = 1  # Keep at 1
    
    # ACO specific
    ACO_ANTS: int = 4  # Minimal ants
    ACO_ITERATIONS: int = 2  # Just 1 iteration
    ACO_ALPHA: float = 1.0
    ACO_BETA: float = 2.0
    ACO_EVAPORATION: float = 0.5
    ACO_FEATURE_FRACTION: float = 0.5
    
    # SA specific
    SA_INITIAL_TEMP: float = 10.0
    SA_COOLING_RATE: float = 0.5  # Faster cooling
    SA_MIN_TEMP: float = 1.0
    SA_ITERATIONS: int = 2
    SKIP_PHASE2: bool = False  # Run Phase 2 (required)
    
    # XAI
    XAI_SAMPLES_PER_CLASS: int = 1  # Minimal for speed
    GRADCAM_TARGET_LAYER: str = "auto"
    SHAP_BACKGROUND_SIZE: int = 10  # Minimal background
    SHAP_NSAMPLES: int = 20  # Minimal samples
    
    # Reproducibility
    SEED: int = 42
    
    # Subset for TEST_RUN
    TEST_RUN_MAX_IMAGES: int = 500
    TEST_RUN_IMAGES_PER_CLASS: int = 100
    
    # NEW: Balanced subset for full run (10,000 images)
    USE_BALANCED_SUBSET: bool = True  # Enable balanced 10k subset
    BALANCED_SUBSET_CONFIG: Dict = None  # Will be set in __post_init__

    def __post_init__(self):
        """Apply TEST_RUN overrides and set balanced subset config."""
        # Set balanced subset configuration (7200 images total)
        # Class distribution: Mild=1800, Moderate=488(all), Non=2912, VeryMild=2000
        self.BALANCED_SUBSET_CONFIG = {
            'Mild Dementia': 1800,
            'Moderate Dementia': 488,  # Use all available
            'Non Demented': 2912,
            'Very mild Dementia': 2000
        }
        
        if self.TEST_RUN:
            self.EPOCHS = 1
            self.META_POPULATION = 4  # Minimum 4 for DE algorithm
            self.META_ITERATIONS = 1
            self.META_EVAL_EPOCHS = 1
            self.ACO_ANTS = 4
            self.ACO_ITERATIONS = 1
            self.EARLY_STOPPING_PATIENCE = 2
            self.SA_ITERATIONS = 1
            self.XAI_SAMPLES_PER_CLASS = 2
            self.SHAP_BACKGROUND_SIZE = 10
            self.SHAP_NSAMPLES = 20
            self.USE_BALANCED_SUBSET = False  # Don't use balanced subset in test mode
        else:
            # Full run mode - use balanced subset
            pass

    def to_dict(self) -> Dict:
        result = asdict(self)
        # Convert BALANCED_SUBSET_CONFIG to serializable format
        return result

    def save(self, path: str):
        with open(path, 'w') as f:
            json.dump(self.to_dict(), f, indent=2)

    @classmethod
    def load(cls, path: str) -> 'Config':
        with open(path, 'r') as f:
            data = json.load(f)
        return cls(**data)

def create_run_directory(base_dir: str) -> str:
    """Create timestamped run directory."""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_id = hashlib.md5(str(time.time()).encode()).hexdigest()[:8]
    run_dir = os.path.join(base_dir, f"{timestamp}_{run_id}")
    subdirs = ['checkpoints', 'logs', 'xai_outputs', 'plots']
    for subdir in subdirs:
        os.makedirs(os.path.join(run_dir, subdir), exist_ok=True)
    return run_dir

# ============================================================
# CONFIGURATION - MODIFY THESE SETTINGS AS NEEDED
# ============================================================
TEST_RUN = False  # Set to True for quick test, False for full training

config = Config(TEST_RUN=TEST_RUN)
set_global_seed(config.SEED)
device = get_device("auto")

print("=" * 50)
print("CONFIGURATION")
print("=" * 50)
print(f"TEST_RUN: {config.TEST_RUN}")
print(f"DATA_DIR: {config.DATA_DIR}")
print(f"OUTPUT_DIR: {config.OUTPUT_DIR}")
print(f"EPOCHS: {config.EPOCHS}")
print(f"BATCH_SIZE: {config.BATCH_SIZE}")
print(f"Device: {device}")
print("=" * 50)

CONFIGURATION
TEST_RUN: False
DATA_DIR: /kaggle/input/imagesoasis
OUTPUT_DIR: /kaggle/working/runs
EPOCHS: 2
BATCH_SIZE: 64
Device: cuda


In [5]:
# Cell 5: Dataset & DataLoader Classes

def find_data_directory(base_path: str) -> str:
    """Find the actual directory containing class folders."""
    possible_paths = [
        base_path,
        os.path.join(base_path, "Data"),
        os.path.join(base_path, "dataset"),
        os.path.join(base_path, "train"),
        os.path.join(base_path, "Dataset"),
    ]
    for path in possible_paths:
        if os.path.exists(path):
            subdirs = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
            if any('demented' in d.lower() for d in subdirs):
                return path
            if 'train' in subdirs:
                train_path = os.path.join(path, 'train')
                return train_path
    for root, dirs, files in os.walk(base_path):
        if any('demented' in d.lower() for d in dirs):
            return root
    return base_path

class OASISDataset(Dataset):
    """Custom dataset for OASIS Alzheimer MRI images."""
    def __init__(self, image_paths: List[str], labels: List[int], transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Error loading {img_path}: {e}")
            image = Image.new('RGB', (224, 224), color='black')
        if self.transform:
            image = self.transform(image)
        return image, label

def load_dataset(config: Config, data_dir: str) -> Tuple[Dict, Dict, Dict]:
    """Load and split dataset."""
    print("\n" + "="*50)
    print("LOADING DATASET")
    print("="*50)
    data_dir = find_data_directory(data_dir)
    print(f"Using data directory: {data_dir}")

    image_paths = []
    labels = []
    class_to_idx = {}
    subdirs = sorted([d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))])
    print(f"Found subdirectories: {subdirs}")

    idx = 0
    for subdir in subdirs:
        subdir_lower = subdir.lower().replace('_', ' ').replace('-', ' ')
        if 'demented' in subdir_lower or 'dementia' in subdir_lower:
            class_to_idx[subdir] = idx
            idx += 1
    if not class_to_idx:
        class_to_idx = {d: i for i, d in enumerate(subdirs)}
    print(f"Class mapping: {class_to_idx}")

    idx_to_class = {v: k for k, v in class_to_idx.items()}

    for class_name, class_idx in class_to_idx.items():
        class_dir = os.path.join(data_dir, class_name)
        if not os.path.isdir(class_dir):
            continue
        for img_name in os.listdir(class_dir):
            if img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                img_path = os.path.join(class_dir, img_name)
                image_paths.append(img_path)
                labels.append(class_idx)

    print(f"\nTotal images found: {len(image_paths)}")
    class_counts = Counter(labels)
    print("\nClass distribution:")
    for idx, count in sorted(class_counts.items()):
        print(f"  {idx_to_class.get(idx, idx)}: {count}")

    if config.TEST_RUN:
        print(f"\nTEST_RUN mode: limiting to {config.TEST_RUN_MAX_IMAGES} images")
        subset_paths = []
        subset_labels = []
        for class_idx in class_to_idx.values():
            class_mask = [i for i, l in enumerate(labels) if l == class_idx]
            n_take = min(len(class_mask), config.TEST_RUN_IMAGES_PER_CLASS)
            if n_take > 0:
                selected = random.sample(class_mask, n_take)
                for i in selected:
                    subset_paths.append(image_paths[i])
                    subset_labels.append(labels[i])
        image_paths = subset_paths
        labels = subset_labels
        print(f"Subset size: {len(image_paths)}")
    
    # Apply balanced subset for full run (10,000 images)
    elif config.USE_BALANCED_SUBSET and config.BALANCED_SUBSET_CONFIG:
        print(f"\nApplying balanced subset (target: 10,000 images)")
        subset_paths = []
        subset_labels = []
        
        for class_name, class_idx in class_to_idx.items():
            class_mask = [i for i, l in enumerate(labels) if l == class_idx]
            
            # Get target count for this class
            target_count = config.BALANCED_SUBSET_CONFIG.get(class_name, len(class_mask))
            n_take = min(len(class_mask), target_count)
            
            if n_take > 0:
                selected = random.sample(class_mask, n_take)
                for i in selected:
                    subset_paths.append(image_paths[i])
                    subset_labels.append(labels[i])
            
            print(f"  {class_name}: {n_take} images (target: {target_count}, available: {len(class_mask)})")
        
        image_paths = subset_paths
        labels = subset_labels
        
        # Recalculate class distribution
        class_counts = Counter(labels)
        print(f"\nBalanced subset - Total: {len(image_paths)} images")
        print("New class distribution:")
        for idx, count in sorted(class_counts.items()):
            print(f"  {idx_to_class.get(idx, idx)}: {count}")

    if not config.TEST_RUN and len(image_paths) < 7000:
        print(f"Note: Using {len(image_paths)} samples for training.")

    train_paths, temp_paths, train_labels, temp_labels = train_test_split(
        image_paths, labels, test_size=0.3, stratify=labels, random_state=config.SEED
    )
    val_paths, test_paths, val_labels, test_labels = train_test_split(
        temp_paths, temp_labels, test_size=0.5, stratify=temp_labels, random_state=config.SEED
    )

    print(f"\nSplit sizes:")
    print(f"  Train: {len(train_paths)}")
    print(f"  Val: {len(val_paths)}")
    print(f"  Test: {len(test_paths)}")

    normalize = T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    aug_strength = config.AUGMENT_STRENGTH

    train_transform = T.Compose([
        T.Resize((config.IMG_SIZE + 32, config.IMG_SIZE + 32)),
        T.RandomResizedCrop(config.IMG_SIZE, scale=(0.8, 1.0)),
        T.RandomHorizontalFlip(p=0.5 * aug_strength),
        T.RandomRotation(degrees=int(15 * aug_strength)),
        T.ColorJitter(
            brightness=0.2 * aug_strength,
            contrast=0.2 * aug_strength,
            saturation=0.1 * aug_strength
        ),
        T.ToTensor(),
        normalize
    ])

    val_transform = T.Compose([
        T.Resize((config.IMG_SIZE, config.IMG_SIZE)),
        T.ToTensor(),
        normalize
    ])

    train_dataset = OASISDataset(train_paths, train_labels, train_transform)
    val_dataset = OASISDataset(val_paths, val_labels, val_transform)
    test_dataset = OASISDataset(test_paths, test_labels, val_transform)

    class_counts_train = Counter(train_labels)
    total_train = len(train_labels)
    class_weights = torch.zeros(len(class_to_idx))
    for idx in range(len(class_to_idx)):
        count = class_counts_train.get(idx, 1)
        class_weights[idx] = total_train / (len(class_to_idx) * count)

    sample_weights = [class_weights[l].item() for l in train_labels]

    datasets = {
        'train': train_dataset,
        'val': val_dataset,
        'test': test_dataset
    }

    metadata = {
        'class_to_idx': class_to_idx,
        'idx_to_class': idx_to_class,
        'class_weights': class_weights,
        'sample_weights': sample_weights,
        'class_counts': dict(class_counts),
        'train_paths': train_paths,
        'train_labels': train_labels,
        'val_paths': val_paths,
        'val_labels': val_labels,
        'test_paths': test_paths,
        'test_labels': test_labels
    }

    transforms = {
        'train': train_transform,
        'val': val_transform,
        'test': val_transform
    }

    return datasets, metadata, transforms

def create_dataloaders(datasets: Dict, metadata: Dict, config: Config) -> Dict[str, DataLoader]:
    """Create data loaders with optional weighted sampling."""
    loaders = {}

    if config.USE_WEIGHTED_SAMPLER and 'sample_weights' in metadata:
        sampler = WeightedRandomSampler(
            weights=metadata['sample_weights'],
            num_samples=len(metadata['sample_weights']),
            replacement=True
        )
        loaders['train'] = DataLoader(
            datasets['train'],
            batch_size=config.BATCH_SIZE,
            sampler=sampler,
            num_workers=config.NUM_WORKERS,
            pin_memory=True
        )
    else:
        loaders['train'] = DataLoader(
            datasets['train'],
            batch_size=config.BATCH_SIZE,
            shuffle=True,
            num_workers=config.NUM_WORKERS,
            pin_memory=True
        )

    loaders['val'] = DataLoader(
        datasets['val'],
        batch_size=config.BATCH_SIZE,
        shuffle=False,
        num_workers=config.NUM_WORKERS,
        pin_memory=True
    )

    loaders['test'] = DataLoader(
        datasets['test'],
        batch_size=config.BATCH_SIZE,
        shuffle=False,
        num_workers=config.NUM_WORKERS,
        pin_memory=True
    )

    return loaders

print("Dataset and DataLoader classes defined.")

Dataset and DataLoader classes defined.


In [6]:
# Cell 6: Model Definition

class FocalLoss(nn.Module):
    """Focal Loss for imbalanced classification."""
    def __init__(self, alpha: Optional[torch.Tensor] = None, gamma: float = 2.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
        ce_loss = F.cross_entropy(inputs, targets, weight=self.alpha, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss
        return focal_loss.mean()

class AlzheimerClassifier(nn.Module):
    """EfficientNet-B0 based classifier for Alzheimer's classification."""
    def __init__(self, num_classes: int = 4, dropout: float = 0.3, pretrained: bool = True):
        super().__init__()
        if TIMM_AVAILABLE:
            self.backbone = timm.create_model('efficientnet_b0', pretrained=pretrained)
            if hasattr(self.backbone, 'classifier'):
                in_features = self.backbone.classifier.in_features
                self.backbone.classifier = nn.Identity()
            elif hasattr(self.backbone, 'fc'):
                in_features = self.backbone.fc.in_features
                self.backbone.fc = nn.Identity()
            else:
                in_features = self.backbone.num_features
        else:
            self.backbone = tv_models.efficientnet_b0(
                weights=tv_models.EfficientNet_B0_Weights.IMAGENET1K_V1 if pretrained else None
            )
            in_features = self.backbone.classifier[1].in_features
            self.backbone.classifier = nn.Identity()

        self.classifier = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(in_features, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout / 2),
            nn.Linear(256, num_classes)
        )
        self.in_features = in_features
        self._feature_mask = None

    def set_feature_mask(self, mask: Optional[torch.Tensor]):
        """Set feature mask for ACO feature selection."""
        self._feature_mask = mask

    def get_features(self, x: torch.Tensor) -> torch.Tensor:
        """Extract backbone features."""
        return self.backbone(x)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        features = self.backbone(x)
        if self._feature_mask is not None:
            features = features * self._feature_mask.to(features.device)
        return self.classifier(features)

    def freeze_backbone(self):
        """Freeze backbone parameters."""
        for param in self.backbone.parameters():
            param.requires_grad = False

    def unfreeze_backbone(self):
        """Unfreeze backbone parameters."""
        for param in self.backbone.parameters():
            param.requires_grad = True

def count_parameters(model: nn.Module) -> Tuple[int, int]:
    """Count total and trainable parameters."""
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total, trainable

def create_model(config: Config, device: torch.device) -> nn.Module:
    """Create and initialize model (single GPU)."""
    model = AlzheimerClassifier(
        num_classes=NUM_CLASSES,
        dropout=config.DROPOUT,
        pretrained=config.PRETRAINED
    )
    total, trainable = count_parameters(model)
    print(f"\nModel created:")
    print(f"  Total parameters: {total:,}")
    print(f"  Trainable parameters: {trainable:,}")
    
    model = model.to(device)
    
    return model

print("Model classes defined.")    

Model classes defined.


In [7]:
# Cell 7: Training & Evaluation Functions

def train_one_epoch(
    model: nn.Module,
    loader: DataLoader,
    criterion: nn.Module,
    optimizer: torch.optim.Optimizer,
    device: torch.device,
    scaler: Optional[GradScaler] = None,
    use_amp: bool = True
) -> Dict[str, float]:
    """Train for one epoch."""
    model.train()
    total_loss = 0.0
    all_preds = []
    all_labels = []
    num_batches = len(loader)

    for batch_idx, (images, labels) in enumerate(loader):
        if batch_idx == 0:
            print(f"    First batch loaded, starting training...", flush=True)
        
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        if use_amp and scaler is not None:
            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        total_loss += loss.item()

        preds = outputs.argmax(dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())
        
        # Progress indicator every 10 batches
        if (batch_idx + 1) % 10 == 0 or batch_idx == num_batches - 1:
            print(f"    Batch {batch_idx + 1}/{num_batches}, Loss: {loss.item():.4f}", flush=True)

    avg_loss = total_loss / len(loader)
    accuracy = accuracy_score(all_labels, all_preds)
    balanced_acc = balanced_accuracy_score(all_labels, all_preds)
    macro_f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)

    return {
        'loss': avg_loss,
        'accuracy': accuracy,
        'balanced_accuracy': balanced_acc,
        'macro_f1': macro_f1
    }

def evaluate(
    model: nn.Module,
    loader: DataLoader,
    criterion: nn.Module,
    device: torch.device,
    use_amp: bool = True
) -> Dict[str, Any]:
    """Evaluate model on a dataset."""
    model.eval()
    total_loss = 0.0
    all_preds = []
    all_labels = []
    all_probs = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)

            if use_amp:
                with autocast():
                    outputs = model(images)
                    loss = criterion(outputs, labels)
            else:
                outputs = model(images)
                loss = criterion(outputs, labels)

            total_loss += loss.item()
            probs = F.softmax(outputs, dim=1).cpu().numpy()
            preds = outputs.argmax(dim=1).cpu().numpy()
            all_probs.extend(probs)
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    avg_loss = total_loss / len(loader)
    accuracy = accuracy_score(all_labels, all_preds)
    balanced_acc = balanced_accuracy_score(all_labels, all_preds)
    macro_f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    precision = precision_score(all_labels, all_preds, average=None, zero_division=0)
    recall = recall_score(all_labels, all_preds, average=None, zero_division=0)
    f1_per_class = f1_score(all_labels, all_preds, average=None, zero_division=0)
    cm = confusion_matrix(all_labels, all_preds)

    return {
        'loss': avg_loss,
        'accuracy': accuracy,
        'balanced_accuracy': balanced_acc,
        'macro_f1': macro_f1,
        'precision_per_class': precision.tolist(),
        'recall_per_class': recall.tolist(),
        'f1_per_class': f1_per_class.tolist(),
        'confusion_matrix': cm.tolist(),
        'predictions': all_preds,
        'labels': all_labels,
        'probabilities': np.array(all_probs)
    }

def predict_proba(
    model: nn.Module,
    loader: DataLoader,
    device: torch.device
) -> Tuple[np.ndarray, np.ndarray]:
    """Get prediction probabilities."""
    model.eval()
    all_probs = []
    all_labels = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs = model(images)
            probs = F.softmax(outputs, dim=1).cpu().numpy()
            all_probs.extend(probs)
            all_labels.extend(labels.numpy())

    return np.array(all_probs), np.array(all_labels)

def train_model(
    model: nn.Module,
    loaders: Dict[str, DataLoader],
    config: Config,
    metadata: Dict,
    device: torch.device,
    run_dir: str,
    epochs: Optional[int] = None,
    verbose: bool = True
) -> Tuple[nn.Module, Dict, List[Dict]]:
    """Full training loop with early stopping."""
    epochs = epochs or config.EPOCHS

    class_weights = metadata['class_weights'].to(device) if config.USE_CLASS_WEIGHTS else None
    if config.USE_FOCAL_LOSS:
        criterion = FocalLoss(alpha=class_weights, gamma=config.FOCAL_LOSS_GAMMA)
    else:
        criterion = nn.CrossEntropyLoss(weight=class_weights)

    if config.OPTIMIZER.lower() == 'adamw':
        optimizer = torch.optim.AdamW(
            model.parameters(),
            lr=config.LR,
            weight_decay=config.WEIGHT_DECAY
        )
    else:
        optimizer = torch.optim.Adam(
            model.parameters(),
            lr=config.LR,
            weight_decay=config.WEIGHT_DECAY
        )

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=2
    )

    scaler = GradScaler() if config.USE_AMP and device.type == 'cuda' else None

    history = []
    best_val_f1 = 0.0
    best_epoch = 0
    patience_counter = 0
    best_model_state = None

    if config.FREEZE_BACKBONE_EPOCHS > 0:
        model.freeze_backbone()
        if verbose:
            print(f"Backbone frozen for first {config.FREEZE_BACKBONE_EPOCHS} epochs")

    for epoch in range(epochs):
        start_time = time.time()
        if verbose:
            print(f"\n  Starting Epoch {epoch + 1}/{epochs}...", flush=True)

        if epoch == config.FREEZE_BACKBONE_EPOCHS:
            model.unfreeze_backbone()
            if verbose:
                print("Backbone unfrozen")

        train_metrics = train_one_epoch(
            model, loaders['train'], criterion, optimizer, device,
            scaler=scaler, use_amp=config.USE_AMP
        )

        val_metrics = evaluate(
            model, loaders['val'], criterion, device, use_amp=config.USE_AMP
        )

        epoch_time = time.time() - start_time
        scheduler.step(val_metrics['macro_f1'])

        history.append({
            'epoch': epoch + 1,
            'train_loss': train_metrics['loss'],
            'train_acc': train_metrics['accuracy'],
            'train_f1': train_metrics['macro_f1'],
            'val_loss': val_metrics['loss'],
            'val_acc': val_metrics['accuracy'],
            'val_balanced_acc': val_metrics['balanced_accuracy'],
            'val_f1': val_metrics['macro_f1'],
            'epoch_time': epoch_time,
            'lr': optimizer.param_groups[0]['lr']
        })

        if verbose:
            print(f"Epoch {epoch+1}/{epochs} ({epoch_time:.1f}s) - "
                  f"Train Loss: {train_metrics['loss']:.4f}, "
                  f"Val F1: {val_metrics['macro_f1']:.4f}, "
                  f"Val Acc: {val_metrics['accuracy']:.4f}")

        # Memory cleanup
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()

        if val_metrics['macro_f1'] > best_val_f1:
            best_val_f1 = val_metrics['macro_f1']
            best_epoch = epoch + 1
            best_model_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            patience_counter = 0
            checkpoint_path = os.path.join(run_dir, 'checkpoints', 'best.pt')
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_f1': best_val_f1,
                'config': config.to_dict()
            }, checkpoint_path)
        else:
            patience_counter += 1

        if patience_counter >= config.EARLY_STOPPING_PATIENCE:
            if verbose:
                print(f"Early stopping at epoch {epoch + 1}")
            break

    if best_model_state is not None:
        model.load_state_dict(best_model_state)

    test_metrics = evaluate(model, loaders['test'], criterion, device, use_amp=config.USE_AMP)

    final_metrics = {
        'best_epoch': best_epoch,
        'best_val_f1': best_val_f1,
        'test_accuracy': test_metrics['accuracy'],
        'test_balanced_accuracy': test_metrics['balanced_accuracy'],
        'test_macro_f1': test_metrics['macro_f1'],
        'test_f1_per_class': test_metrics['f1_per_class'],
        'test_confusion_matrix': test_metrics['confusion_matrix']
    }

    save_training_curves(history, run_dir)
    pd.DataFrame(history).to_csv(
        os.path.join(run_dir, 'logs', 'training_history.csv'),
        index=False
    )

    return model, final_metrics, history

def save_training_curves(history: List[Dict], run_dir: str):
    """Save training curves as plots."""
    df = pd.DataFrame(history)
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))

    axes[0, 0].plot(df['epoch'], df['train_loss'], label='Train')
    axes[0, 0].plot(df['epoch'], df['val_loss'], label='Val')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].set_title('Loss Curves')
    axes[0, 0].legend()
    axes[0, 0].grid(True)

    axes[0, 1].plot(df['epoch'], df['train_acc'], label='Train')
    axes[0, 1].plot(df['epoch'], df['val_acc'], label='Val')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Accuracy')
    axes[0, 1].set_title('Accuracy Curves')
    axes[0, 1].legend()
    axes[0, 1].grid(True)

    axes[1, 0].plot(df['epoch'], df['train_f1'], label='Train')
    axes[1, 0].plot(df['epoch'], df['val_f1'], label='Val')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Macro F1')
    axes[1, 0].set_title('F1 Score Curves')
    axes[1, 0].legend()
    axes[1, 0].grid(True)

    axes[1, 1].plot(df['epoch'], df['lr'])
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Learning Rate')
    axes[1, 1].set_title('Learning Rate Schedule')
    axes[1, 1].grid(True)

    plt.tight_layout()
    plt.savefig(os.path.join(run_dir, 'plots', 'training_curves.png'), dpi=150)
    plt.close()

print("Training & Evaluation functions defined.")

Training & Evaluation functions defined.


In [8]:
# Cell 8b: Visualization & Comparison Plots (NEW)

def plot_algorithm_comparison(results_df: pd.DataFrame, run_dir: str, title: str = "Phase 1"):
    """Create bar chart comparing algorithm performance."""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Sort by score for better visualization
    df_sorted = results_df.sort_values('best_score', ascending=False)
    algorithms = df_sorted['algorithm'].tolist()
    scores = df_sorted['best_score'].tolist()
    times = df_sorted['time_seconds'].tolist()
    
    # Color palette
    colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(algorithms)))
    
    # Plot 1: Scores
    bars1 = axes[0].bar(algorithms, scores, color=colors, edgecolor='black', linewidth=1.2)
    axes[0].set_xlabel('Algorithm', fontsize=12)
    axes[0].set_ylabel('Best Score (Macro F1)', fontsize=12)
    axes[0].set_title(f'{title}: Algorithm Performance Comparison', fontsize=14, fontweight='bold')
    axes[0].set_ylim(0, max(scores) * 1.15 if scores else 1)
    axes[0].grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for bar, score in zip(bars1, scores):
        axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                     f'{score:.4f}', ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    # Highlight best algorithm
    if scores:
        best_idx = scores.index(max(scores))
        bars1[best_idx].set_edgecolor('gold')
        bars1[best_idx].set_linewidth(3)
    
    # Plot 2: Time
    bars2 = axes[1].bar(algorithms, times, color=colors, edgecolor='black', linewidth=1.2)
    axes[1].set_xlabel('Algorithm', fontsize=12)
    axes[1].set_ylabel('Time (seconds)', fontsize=12)
    axes[1].set_title(f'{title}: Algorithm Execution Time', fontsize=14, fontweight='bold')
    axes[1].grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for bar, t in zip(bars2, times):
        axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                     f'{t:.1f}s', ha='center', va='bottom', fontsize=10)
    
    plt.tight_layout()
    filename = f'{title.lower().replace(" ", "_")}_algorithm_comparison.png'
    plt.savefig(os.path.join(run_dir, 'plots', filename), dpi=150, bbox_inches='tight')
    plt.close()
    print(f"  Saved: plots/{filename}")


def plot_convergence_curves(algo_histories: Dict[str, List[float]], run_dir: str, title: str = "Convergence"):
    """Plot convergence curves for multiple algorithms."""
    if not algo_histories:
        return
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    colors = plt.cm.tab10(np.linspace(0, 1, len(algo_histories)))
    markers = ['o', 's', '^', 'D', 'v', '<', '>', 'p']
    
    for i, (algo_name, history) in enumerate(algo_histories.items()):
        if history:
            iterations = range(1, len(history) + 1)
            ax.plot(iterations, history, label=algo_name, color=colors[i],
                   marker=markers[i % len(markers)], markersize=6, linewidth=2)
    
    ax.set_xlabel('Iteration', fontsize=12)
    ax.set_ylabel('Best Score (Macro F1)', fontsize=12)
    ax.set_title(f'{title} Curves', fontsize=14, fontweight='bold')
    ax.legend(loc='lower right', fontsize=10)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    filename = f'{title.lower().replace(" ", "_")}_curves.png'
    plt.savefig(os.path.join(run_dir, 'plots', filename), dpi=150, bbox_inches='tight')
    plt.close()
    print(f"  Saved: plots/{filename}")


def plot_metaheuristic_ranking(rankings_df: pd.DataFrame, run_dir: str):
    """Create comprehensive ranking visualization."""
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Prepare data
    df = rankings_df.sort_values('rank').reset_index(drop=True)
    algorithms = df['algorithm'].tolist()
    scores = df['score'].tolist()
    times = df['time'].tolist()
    efficiency = df['efficiency'].tolist() if 'efficiency' in df.columns else [s/(t+1e-6) for s, t in zip(scores, times)]
    
    colors = plt.cm.RdYlGn(np.linspace(0.2, 0.8, len(algorithms)))[::-1]  # Green for best
    
    # Plot 1: Horizontal bar chart of scores (ranked)
    bars1 = axes[0, 0].barh(algorithms[::-1], scores[::-1], color=colors[::-1], edgecolor='black')
    axes[0, 0].set_xlabel('Score (Macro F1)', fontsize=11)
    axes[0, 0].set_title('Algorithm Ranking by Score', fontsize=12, fontweight='bold')
    axes[0, 0].grid(axis='x', alpha=0.3)
    for bar, score in zip(bars1, scores[::-1]):
        axes[0, 0].text(bar.get_width() + 0.005, bar.get_y() + bar.get_height()/2,
                        f'{score:.4f}', va='center', fontsize=9)
    
    # Plot 2: Scatter plot (Score vs Time)
    scatter = axes[0, 1].scatter(times, scores, c=range(len(algorithms)), cmap='RdYlGn_r', 
                                  s=200, edgecolor='black', linewidth=1.5)
    for i, algo in enumerate(algorithms):
        axes[0, 1].annotate(algo, (times[i], scores[i]), xytext=(5, 5), 
                            textcoords='offset points', fontsize=9)
    axes[0, 1].set_xlabel('Time (seconds)', fontsize=11)
    axes[0, 1].set_ylabel('Score (Macro F1)', fontsize=11)
    axes[0, 1].set_title('Score vs Execution Time', fontsize=12, fontweight='bold')
    axes[0, 1].grid(True, alpha=0.3)
    
    # Plot 3: Efficiency bar chart
    eff_sorted_idx = np.argsort(efficiency)[::-1]
    eff_algos = [algorithms[i] for i in eff_sorted_idx]
    eff_vals = [efficiency[i] for i in eff_sorted_idx]
    eff_colors = plt.cm.coolwarm(np.linspace(0.2, 0.8, len(algorithms)))
    
    bars3 = axes[1, 0].bar(eff_algos, eff_vals, color=eff_colors, edgecolor='black')
    axes[1, 0].set_xlabel('Algorithm', fontsize=11)
    axes[1, 0].set_ylabel('Efficiency (Score/Time)', fontsize=11)
    axes[1, 0].set_title('Algorithm Efficiency', fontsize=12, fontweight='bold')
    axes[1, 0].tick_params(axis='x', rotation=45)
    axes[1, 0].grid(axis='y', alpha=0.3)
    
    # Plot 4: Summary table as text
    axes[1, 1].axis('off')
    table_data = []
    for i, (algo, score, t, eff) in enumerate(zip(algorithms, scores, times, efficiency)):
        table_data.append([i+1, algo, f'{score:.4f}', f'{t:.1f}s', f'{eff:.4f}'])
    
    table = axes[1, 1].table(
        cellText=table_data,
        colLabels=['Rank', 'Algorithm', 'Score', 'Time', 'Efficiency'],
        loc='center',
        cellLoc='center',
        colColours=['lightblue'] * 5
    )
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.5)
    axes[1, 1].set_title('Summary Table', fontsize=12, fontweight='bold', y=0.9)
    
    plt.tight_layout()
    plt.savefig(os.path.join(run_dir, 'plots', 'metaheuristic_ranking.png'), dpi=150, bbox_inches='tight')
    plt.close()
    print("  Saved: plots/metaheuristic_ranking.png")


def plot_baseline_vs_optimized(baseline_metrics: Dict, final_metrics: Dict, run_dir: str):
    """Create comparison plot between baseline and optimized model."""
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Metrics to compare
    metrics = ['test_accuracy', 'test_balanced_accuracy', 'test_macro_f1']
    labels = ['Accuracy', 'Balanced Acc', 'Macro F1']
    
    baseline_vals = [baseline_metrics.get(m, 0) for m in metrics]
    optimized_vals = [final_metrics.get(m, 0) for m in metrics]
    
    x = np.arange(len(labels))
    width = 0.35
    
    # Plot 1: Grouped bar chart
    bars1 = axes[0].bar(x - width/2, baseline_vals, width, label='Baseline', color='#FF6B6B', edgecolor='black')
    bars2 = axes[0].bar(x + width/2, optimized_vals, width, label='Optimized', color='#4ECDC4', edgecolor='black')
    
    axes[0].set_ylabel('Score', fontsize=12)
    axes[0].set_title('Baseline vs Optimized Performance', fontsize=14, fontweight='bold')
    axes[0].set_xticks(x)
    axes[0].set_xticklabels(labels, fontsize=11)
    axes[0].legend(loc='lower right', fontsize=10)
    axes[0].set_ylim(0, 1.1)
    axes[0].grid(axis='y', alpha=0.3)
    
    # Add value labels
    for bar in bars1:
        axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
                     f'{bar.get_height():.3f}', ha='center', va='bottom', fontsize=9)
    for bar in bars2:
        axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
                     f'{bar.get_height():.3f}', ha='center', va='bottom', fontsize=9)
    
    # Plot 2: Improvement chart
    improvements = [(o - b) for b, o in zip(baseline_vals, optimized_vals)]
    colors = ['green' if imp >= 0 else 'red' for imp in improvements]
    
    bars3 = axes[1].bar(labels, improvements, color=colors, edgecolor='black', linewidth=1.2)
    axes[1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    axes[1].set_ylabel('Improvement', fontsize=12)
    axes[1].set_title('Performance Improvement (Optimized - Baseline)', fontsize=14, fontweight='bold')
    axes[1].grid(axis='y', alpha=0.3)
    
    # Add value labels
    for bar, imp in zip(bars3, improvements):
        va = 'bottom' if imp >= 0 else 'top'
        offset = 0.01 if imp >= 0 else -0.01
        axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + offset,
                     f'{imp:+.4f}', ha='center', va=va, fontsize=10, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(os.path.join(run_dir, 'plots', 'baseline_vs_optimized.png'), dpi=150, bbox_inches='tight')
    plt.close()
    print("  Saved: plots/baseline_vs_optimized.png")


def plot_xai_algorithm_comparison(xai_results: Dict, run_dir: str):
    """Create comparison plot for XAI optimization algorithms."""
    if not xai_results:
        return
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Grad-CAM comparison
    if 'gradcam' in xai_results and xai_results['gradcam']:
        gradcam_data = xai_results['gradcam']
        algos = list(gradcam_data.keys())
        scores = [gradcam_data[a].get('score', 0) for a in algos]
        
        colors = plt.cm.plasma(np.linspace(0.2, 0.8, len(algos)))
        bars = axes[0].bar(algos, scores, color=colors, edgecolor='black', linewidth=1.2)
        axes[0].set_xlabel('Algorithm', fontsize=12)
        axes[0].set_ylabel('XAI Score', fontsize=12)
        axes[0].set_title('Grad-CAM Optimization: Algorithm Comparison', fontsize=13, fontweight='bold')
        axes[0].tick_params(axis='x', rotation=45)
        axes[0].grid(axis='y', alpha=0.3)
        
        for bar, score in zip(bars, scores):
            axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                         f'{score:.4f}', ha='center', va='bottom', fontsize=9, fontweight='bold')
        
        # Highlight best
        if scores:
            best_idx = scores.index(max(scores))
            bars[best_idx].set_edgecolor('gold')
            bars[best_idx].set_linewidth(3)
    else:
        axes[0].text(0.5, 0.5, 'No Grad-CAM data', ha='center', va='center', fontsize=12)
        axes[0].set_title('Grad-CAM Optimization', fontsize=13, fontweight='bold')
    
    # SHAP comparison (if available)
    if 'shap' in xai_results and xai_results['shap']:
        shap_data = xai_results['shap']
        algos = list(shap_data.keys())
        scores = [shap_data[a].get('score', 0) for a in algos]
        
        colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(algos)))
        bars = axes[1].bar(algos, scores, color=colors, edgecolor='black', linewidth=1.2)
        axes[1].set_xlabel('Algorithm', fontsize=12)
        axes[1].set_ylabel('XAI Score', fontsize=12)
        axes[1].set_title('SHAP Optimization: Algorithm Comparison', fontsize=13, fontweight='bold')
        axes[1].tick_params(axis='x', rotation=45)
        axes[1].grid(axis='y', alpha=0.3)
        
        for bar, score in zip(bars, scores):
            axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                         f'{score:.4f}', ha='center', va='bottom', fontsize=9, fontweight='bold')
    else:
        axes[1].text(0.5, 0.5, 'No SHAP data\n(SHAP may be disabled)', ha='center', va='center', fontsize=12)
        axes[1].set_title('SHAP Optimization', fontsize=13, fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(os.path.join(run_dir, 'plots', 'xai_algorithm_comparison.png'), dpi=150, bbox_inches='tight')
    plt.close()
    print("  Saved: plots/xai_algorithm_comparison.png")


def plot_confusion_matrix_comparison(baseline_cm: List, optimized_cm: List, class_names: List[str], run_dir: str):
    """Plot confusion matrices for baseline and optimized models side by side."""
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    baseline_cm = np.array(baseline_cm)
    optimized_cm = np.array(optimized_cm)
    
    # Baseline confusion matrix
    im1 = axes[0].imshow(baseline_cm, interpolation='nearest', cmap=plt.cm.Blues)
    axes[0].set_title('Baseline Model', fontsize=14, fontweight='bold')
    plt.colorbar(im1, ax=axes[0], fraction=0.046)
    
    tick_marks = np.arange(len(class_names))
    axes[0].set_xticks(tick_marks)
    axes[0].set_yticks(tick_marks)
    axes[0].set_xticklabels(class_names, rotation=45, ha='right', fontsize=8)
    axes[0].set_yticklabels(class_names, fontsize=8)
    axes[0].set_xlabel('Predicted', fontsize=11)
    axes[0].set_ylabel('True', fontsize=11)
    
    # Add text annotations
    thresh = baseline_cm.max() / 2.
    for i in range(baseline_cm.shape[0]):
        for j in range(baseline_cm.shape[1]):
            axes[0].text(j, i, format(baseline_cm[i, j], 'd'),
                        ha="center", va="center",
                        color="white" if baseline_cm[i, j] > thresh else "black", fontsize=9)
    
    # Optimized confusion matrix
    im2 = axes[1].imshow(optimized_cm, interpolation='nearest', cmap=plt.cm.Greens)
    axes[1].set_title('Optimized Model', fontsize=14, fontweight='bold')
    plt.colorbar(im2, ax=axes[1], fraction=0.046)
    
    axes[1].set_xticks(tick_marks)
    axes[1].set_yticks(tick_marks)
    axes[1].set_xticklabels(class_names, rotation=45, ha='right', fontsize=8)
    axes[1].set_yticklabels(class_names, fontsize=8)
    axes[1].set_xlabel('Predicted', fontsize=11)
    axes[1].set_ylabel('True', fontsize=11)
    
    # Add text annotations
    thresh = optimized_cm.max() / 2.
    for i in range(optimized_cm.shape[0]):
        for j in range(optimized_cm.shape[1]):
            axes[1].text(j, i, format(optimized_cm[i, j], 'd'),
                        ha="center", va="center",
                        color="white" if optimized_cm[i, j] > thresh else "black", fontsize=9)
    
    plt.tight_layout()
    plt.savefig(os.path.join(run_dir, 'plots', 'confusion_matrix_comparison.png'), dpi=150, bbox_inches='tight')
    plt.close()
    print("  Saved: plots/confusion_matrix_comparison.png")

print("Visualization & Comparison Plot functions defined.")

Visualization & Comparison Plot functions defined.


In [9]:
# Cell 8: ACO Feature Selection

class AntColonyOptimization:
    """ACO for feature selection on deep embeddings."""
    def __init__(
        self,
        n_features: int,
        n_ants: int = 10,
        n_iterations: int = 20,
        alpha: float = 1.0,
        beta: float = 2.0,
        evaporation: float = 0.5,
        feature_fraction: float = 0.5,
        seed: int = 42
    ):
        self.n_features = n_features
        self.n_ants = n_ants
        self.n_iterations = n_iterations
        self.alpha = alpha
        self.beta = beta
        self.evaporation = evaporation
        self.feature_fraction = feature_fraction
        self.seed = seed
        np.random.seed(seed)
        self.pheromones = np.ones(n_features)
        self.best_mask = None
        self.best_score = -np.inf
        self.history = []

    def _construct_solution(self) -> np.ndarray:
        """Construct a feature subset using pheromone trails."""
        n_select = max(1, int(self.n_features * self.feature_fraction))
        probs = self.pheromones ** self.alpha
        probs = probs / probs.sum()
        selected = np.random.choice(
            self.n_features,
            size=n_select,
            replace=False,
            p=probs
        )
        mask = np.zeros(self.n_features, dtype=np.float32)
        mask[selected] = 1.0
        return mask

    def _update_pheromones(self, solutions: List[np.ndarray], scores: List[float]):
        """Update pheromone trails based on solutions."""
        self.pheromones *= (1 - self.evaporation)
        for mask, score in zip(solutions, scores):
            if score > 0:
                self.pheromones += mask * score
        if self.best_mask is not None and self.best_score > 0:
            self.pheromones += self.best_mask * self.best_score * 0.5
        self.pheromones = np.clip(self.pheromones, 0.1, 10.0)

    def optimize(self, evaluate_fn: Callable[[np.ndarray], float]) -> Tuple[np.ndarray, float]:
        """Run ACO optimization."""
        print("\nRunning ACO Feature Selection...")
        for iteration in range(self.n_iterations):
            solutions = []
            scores = []
            for ant in range(self.n_ants):
                mask = self._construct_solution()
                score = evaluate_fn(mask)
                solutions.append(mask)
                scores.append(score)
                if score > self.best_score:
                    self.best_score = score
                    self.best_mask = mask.copy()

            self._update_pheromones(solutions, scores)
            self.history.append({
                'iteration': iteration + 1,
                'best_score': self.best_score,
                'mean_score': np.mean(scores),
                'n_selected': int(self.best_mask.sum()) if self.best_mask is not None else 0
            })
            print(f"  Iteration {iteration + 1}/{self.n_iterations}: "
                  f"Best Score = {self.best_score:.4f}, "
                  f"Mean = {np.mean(scores):.4f}")

        return self.best_mask, self.best_score

def extract_features(
    model: nn.Module,
    loader: DataLoader,
    device: torch.device
) -> Tuple[np.ndarray, np.ndarray]:
    """Extract backbone features from the model."""
    model.eval()
    all_features = []
    all_labels = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            features = model.get_features(images)
            all_features.append(features.cpu().numpy())
            all_labels.append(labels.numpy())

    return np.vstack(all_features), np.concatenate(all_labels)

def run_aco_feature_selection(
    model: nn.Module,
    loaders: Dict[str, DataLoader],
    config: Config,
    device: torch.device
) -> Tuple[np.ndarray, Dict]:
    """Run ACO-based feature selection."""
    print("\n" + "="*50)
    print("ACO FEATURE SELECTION")
    print("="*50)
    print("Extracting features...")
    train_features, train_labels = extract_features(model, loaders['train'], device)
    val_features, val_labels = extract_features(model, loaders['val'], device)
    n_features = train_features.shape[1]
    print(f"Feature dimension: {n_features}")

    scaler = StandardScaler()
    train_features_scaled = scaler.fit_transform(train_features)
    val_features_scaled = scaler.transform(val_features)

    baseline_clf = LogisticRegression(max_iter=1000, random_state=config.SEED)
    baseline_clf.fit(train_features_scaled, train_labels)
    baseline_preds = baseline_clf.predict(val_features_scaled)
    baseline_f1 = f1_score(val_labels, baseline_preds, average='macro', zero_division=0)
    print(f"Baseline (all features): Macro F1 = {baseline_f1:.4f}")

    def evaluate_mask(mask: np.ndarray) -> float:
        selected_idx = np.where(mask > 0.5)[0]
        if len(selected_idx) == 0:
            return 0.0
        X_train_sub = train_features_scaled[:, selected_idx]
        X_val_sub = val_features_scaled[:, selected_idx]
        clf = LogisticRegression(max_iter=500, random_state=config.SEED)
        clf.fit(X_train_sub, train_labels)
        preds = clf.predict(X_val_sub)
        return f1_score(val_labels, preds, average='macro', zero_division=0)

    aco = AntColonyOptimization(
        n_features=n_features,
        n_ants=config.ACO_ANTS,
        n_iterations=config.ACO_ITERATIONS,
        alpha=config.ACO_ALPHA,
        beta=config.ACO_BETA,
        evaporation=config.ACO_EVAPORATION,
        feature_fraction=config.ACO_FEATURE_FRACTION,
        seed=config.SEED
    )

    best_mask, best_score = aco.optimize(evaluate_mask)

    n_selected = int(best_mask.sum())
    print(f"\nACO Result: Selected {n_selected}/{n_features} features")
    print(f"Best Macro F1: {best_score:.4f}")
    print(f"Improvement: {best_score - baseline_f1:+.4f}")

    results = {
        'baseline_f1': baseline_f1,
        'aco_best_f1': best_score,
        'n_features_total': n_features,
        'n_features_selected': n_selected,
        'improvement': best_score - baseline_f1,
        'history': aco.history
    }

    return best_mask, results

print("ACO Feature Selection defined.")

ACO Feature Selection defined.


In [10]:
# Cell 9: Metaheuristic Optimizers (DE, GWO, PSO, BAT, WOA, SA)

class HyperparameterSpace:
    """Define hyperparameter search space."""
    BOUNDS = {
        'lr': (1e-5, 3e-3),
        'weight_decay': (0.0, 1e-2),
        'dropout': (0.0, 0.6),
        'unfreeze_epoch': (0, 3),
        'batch_size_idx': (0, 2),
        'augment_strength': (0.5, 1.5)
    }
    BATCH_SIZES = [8, 16, 32]

    @classmethod
    def decode(cls, vector: np.ndarray) -> Dict:
        """Decode continuous vector to hyperparameters."""
        return {
            'lr': 10 ** (np.log10(cls.BOUNDS['lr'][0]) + 
                        vector[0] * (np.log10(cls.BOUNDS['lr'][1]) - np.log10(cls.BOUNDS['lr'][0]))),
            'weight_decay': cls.BOUNDS['weight_decay'][0] + 
                           vector[1] * (cls.BOUNDS['weight_decay'][1] - cls.BOUNDS['weight_decay'][0]),
            'dropout': cls.BOUNDS['dropout'][0] + 
                      vector[2] * (cls.BOUNDS['dropout'][1] - cls.BOUNDS['dropout'][0]),
            'unfreeze_epoch': int(cls.BOUNDS['unfreeze_epoch'][0] + 
                                 vector[3] * (cls.BOUNDS['unfreeze_epoch'][1] - cls.BOUNDS['unfreeze_epoch'][0])),
            'batch_size': cls.BATCH_SIZES[min(int(vector[4] * 3), 2)],
            'augment_strength': cls.BOUNDS['augment_strength'][0] + 
                               vector[5] * (cls.BOUNDS['augment_strength'][1] - cls.BOUNDS['augment_strength'][0])
        }

    @classmethod
    def n_dims(cls) -> int:
        return 6

def evaluate_hyperparams(
    hyperparams: Dict,
    datasets: Dict,
    metadata: Dict,
    config: Config,
    device: torch.device,
    eval_epochs: int
) -> float:
    """Evaluate a set of hyperparameters."""
    eval_config = Config(TEST_RUN=config.TEST_RUN)
    eval_config.LR = hyperparams['lr']
    eval_config.WEIGHT_DECAY = hyperparams['weight_decay']
    eval_config.DROPOUT = hyperparams['dropout']
    eval_config.FREEZE_BACKBONE_EPOCHS = hyperparams['unfreeze_epoch']
    eval_config.BATCH_SIZE = hyperparams['batch_size']
    eval_config.AUGMENT_STRENGTH = hyperparams['augment_strength']
    eval_config.EPOCHS = eval_epochs
    eval_config.EARLY_STOPPING_PATIENCE = max(eval_epochs, 2)
    eval_config.USE_WEIGHTED_SAMPLER = config.USE_WEIGHTED_SAMPLER
    eval_config.USE_CLASS_WEIGHTS = config.USE_CLASS_WEIGHTS

    model = AlzheimerClassifier(
        num_classes=NUM_CLASSES,
        dropout=eval_config.DROPOUT,
        pretrained=True
    ).to(device)

    loaders = create_dataloaders(datasets, metadata, eval_config)
    class_weights = metadata['class_weights'].to(device) if eval_config.USE_CLASS_WEIGHTS else None
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=eval_config.LR,
        weight_decay=eval_config.WEIGHT_DECAY
    )

    if eval_config.FREEZE_BACKBONE_EPOCHS > 0:
        model.freeze_backbone()  # model is AlzheimerClassifier directly (no DataParallel here)

    scaler = GradScaler() if config.USE_AMP and device.type == 'cuda' else None

    best_val_f1 = 0.0
    for epoch in range(eval_epochs):
        if epoch == eval_config.FREEZE_BACKBONE_EPOCHS:
            model.unfreeze_backbone()  # model is AlzheimerClassifier directly (no DataParallel here)
        train_one_epoch(model, loaders['train'], criterion, optimizer, device, scaler)
        val_metrics = evaluate(model, loaders['val'], criterion, device)
        if val_metrics['macro_f1'] > best_val_f1:
            best_val_f1 = val_metrics['macro_f1']

    # Memory cleanup
    del model, optimizer, criterion
    if scaler is not None:
        del scaler
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

    return best_val_f1

class DifferentialEvolution:
    """Differential Evolution optimizer."""
    def __init__(self, n_dims: int, population_size: int = 20, iterations: int = 50,
                 F: float = 0.8, CR: float = 0.9, seed: Optional[int] = 42):
        self.n_dims = n_dims
        self.population_size = population_size
        self.iterations = iterations
        self.F = F
        self.CR = CR
        if seed is not None:
            np.random.seed(seed)
        self.population = np.random.rand(population_size, n_dims)
        self.fitness = np.full(population_size, -np.inf)
        self.best_solution = None
        self.best_fitness = -np.inf
        self.history = []

    def optimize(self, evaluate_fn: Callable[[np.ndarray], float]) -> Tuple[np.ndarray, float]:
        for i in range(self.population_size):
            self.fitness[i] = evaluate_fn(self.population[i])
            if self.fitness[i] > self.best_fitness:
                self.best_fitness = self.fitness[i]
                self.best_solution = self.population[i].copy()

        for iteration in range(self.iterations):
            for i in range(self.population_size):
                candidates = list(range(self.population_size))
                candidates.remove(i)
                a, b, c = np.random.choice(candidates, 3, replace=False)
                mutant = self.population[a] + self.F * (self.population[b] - self.population[c])
                mutant = np.clip(mutant, 0, 1)
                trial = np.copy(self.population[i])
                crossover_mask = np.random.rand(self.n_dims) < self.CR
                if not crossover_mask.any():
                    crossover_mask[np.random.randint(self.n_dims)] = True
                trial[crossover_mask] = mutant[crossover_mask]
                trial_fitness = evaluate_fn(trial)
                if trial_fitness > self.fitness[i]:
                    self.population[i] = trial
                    self.fitness[i] = trial_fitness
                    if trial_fitness > self.best_fitness:
                        self.best_fitness = trial_fitness
                        self.best_solution = trial.copy()

            self.history.append({'iteration': iteration + 1, 'best_fitness': self.best_fitness, 'mean_fitness': np.mean(self.fitness)})
            print(f"  DE Iter {iteration + 1}/{self.iterations}: Best = {self.best_fitness:.4f}")
        return self.best_solution, self.best_fitness

class GrayWolfOptimizer:
    """Gray Wolf Optimizer."""
    def __init__(self, n_dims: int, population_size: int = 20, iterations: int = 50,
                 a_start: float = 2.0, a_end: float = 0.0, seed: Optional[int] = 42):
        self.n_dims = n_dims
        self.population_size = population_size
        self.iterations = iterations
        self.a_start = a_start
        self.a_end = a_end
        if seed is not None:
            np.random.seed(seed)
        self.population = np.random.rand(population_size, n_dims)
        self.fitness = np.full(population_size, -np.inf)
        self.alpha = None
        self.beta = None
        self.delta = None
        self.alpha_score = -np.inf
        self.beta_score = -np.inf
        self.delta_score = -np.inf
        self.history = []

    def optimize(self, evaluate_fn: Callable[[np.ndarray], float]) -> Tuple[np.ndarray, float]:
        for i in range(self.population_size):
            self.fitness[i] = evaluate_fn(self.population[i])
            self._update_leaders(i)

        for iteration in range(self.iterations):
            a = self.a_start - (self.a_start - self.a_end) * (iteration / self.iterations)
            for i in range(self.population_size):
                A1 = 2 * a * np.random.rand(self.n_dims) - a
                C1 = 2 * np.random.rand(self.n_dims)
                D_alpha = np.abs(C1 * self.alpha - self.population[i])
                X1 = self.alpha - A1 * D_alpha
                A2 = 2 * a * np.random.rand(self.n_dims) - a
                C2 = 2 * np.random.rand(self.n_dims)
                D_beta = np.abs(C2 * self.beta - self.population[i])
                X2 = self.beta - A2 * D_beta
                A3 = 2 * a * np.random.rand(self.n_dims) - a
                C3 = 2 * np.random.rand(self.n_dims)
                D_delta = np.abs(C3 * self.delta - self.population[i])
                X3 = self.delta - A3 * D_delta
                self.population[i] = np.clip((X1 + X2 + X3) / 3, 0, 1)

            for i in range(self.population_size):
                self.fitness[i] = evaluate_fn(self.population[i])
                self._update_leaders(i)

            self.history.append({'iteration': iteration + 1, 'best_fitness': self.alpha_score, 'mean_fitness': np.mean(self.fitness)})
            print(f"  GWO Iter {iteration + 1}/{self.iterations}: Best = {self.alpha_score:.4f}")
        return self.alpha, self.alpha_score

    def _update_leaders(self, idx: int):
        score = self.fitness[idx]
        pos = self.population[idx].copy()
        if score > self.alpha_score:
            self.delta_score = self.beta_score
            self.delta = self.beta.copy() if self.beta is not None else pos.copy()
            self.beta_score = self.alpha_score
            self.beta = self.alpha.copy() if self.alpha is not None else pos.copy()
            self.alpha_score = score
            self.alpha = pos
        elif score > self.beta_score:
            self.delta_score = self.beta_score
            self.delta = self.beta.copy() if self.beta is not None else pos.copy()
            self.beta_score = score
            self.beta = pos
        elif score > self.delta_score:
            self.delta_score = score
            self.delta = pos

class ParticleSwarmOptimization:
    """Particle Swarm Optimization."""
    def __init__(self, n_dims: int, population_size: int = 20, iterations: int = 50,
                 w: float = 0.7, c1: float = 1.5, c2: float = 1.5, seed: Optional[int] = 42):
        self.n_dims = n_dims
        self.population_size = population_size
        self.iterations = iterations
        self.w = w
        self.c1 = c1
        self.c2 = c2
        if seed is not None:
            np.random.seed(seed)
        self.positions = np.random.rand(population_size, n_dims)
        self.velocities = np.random.rand(population_size, n_dims) * 0.1
        self.p_best = self.positions.copy()
        self.p_best_scores = np.full(population_size, -np.inf)
        self.g_best = None
        self.g_best_score = -np.inf
        self.history = []

    def optimize(self, evaluate_fn: Callable[[np.ndarray], float]) -> Tuple[np.ndarray, float]:
        for i in range(self.population_size):
            score = evaluate_fn(self.positions[i])
            self.p_best_scores[i] = score
            if score > self.g_best_score:
                self.g_best_score = score
                self.g_best = self.positions[i].copy()

        for iteration in range(self.iterations):
            for i in range(self.population_size):
                r1, r2 = np.random.rand(2)
                self.velocities[i] = (self.w * self.velocities[i] +
                                      self.c1 * r1 * (self.p_best[i] - self.positions[i]) +
                                      self.c2 * r2 * (self.g_best - self.positions[i]))
                self.velocities[i] = np.clip(self.velocities[i], -0.5, 0.5)
                self.positions[i] = np.clip(self.positions[i] + self.velocities[i], 0, 1)
                score = evaluate_fn(self.positions[i])
                if score > self.p_best_scores[i]:
                    self.p_best_scores[i] = score
                    self.p_best[i] = self.positions[i].copy()
                    if score > self.g_best_score:
                        self.g_best_score = score
                        self.g_best = self.positions[i].copy()

            self.history.append({'iteration': iteration + 1, 'best_fitness': self.g_best_score, 'mean_fitness': np.mean(self.p_best_scores)})
            print(f"  PSO Iter {iteration + 1}/{self.iterations}: Best = {self.g_best_score:.4f}")
        return self.g_best, self.g_best_score

class BatAlgorithm:
    """Bat Algorithm optimizer."""
    def __init__(self, n_dims: int, population_size: int = 20, iterations: int = 50,
                 f_min: float = 0.0, f_max: float = 2.0, A_init: float = 0.5, r_init: float = 0.5,
                 alpha: float = 0.9, gamma: float = 0.9, seed: Optional[int] = 42):
        self.n_dims = n_dims
        self.population_size = population_size
        self.iterations = iterations
        self.f_min = f_min
        self.f_max = f_max
        self.alpha = alpha
        self.gamma = gamma
        if seed is not None:
            np.random.seed(seed)
        self.positions = np.random.rand(population_size, n_dims)
        self.velocities = np.zeros((population_size, n_dims))
        self.frequencies = np.zeros(population_size)
        self.A = np.full(population_size, A_init)
        self.r = np.full(population_size, r_init)
        self.r_init = r_init
        self.fitness = np.full(population_size, -np.inf)
        self.best_solution = None
        self.best_fitness = -np.inf
        self.history = []

    def optimize(self, evaluate_fn: Callable[[np.ndarray], float]) -> Tuple[np.ndarray, float]:
        for i in range(self.population_size):
            self.fitness[i] = evaluate_fn(self.positions[i])
            if self.fitness[i] > self.best_fitness:
                self.best_fitness = self.fitness[i]
                self.best_solution = self.positions[i].copy()

        for iteration in range(self.iterations):
            for i in range(self.population_size):
                self.frequencies[i] = self.f_min + (self.f_max - self.f_min) * np.random.rand()
                self.velocities[i] = self.velocities[i] + (self.positions[i] - self.best_solution) * self.frequencies[i]
                new_position = self.positions[i] + self.velocities[i]
                if np.random.rand() > self.r[i]:
                    new_position = self.best_solution + 0.01 * np.random.randn(self.n_dims) * np.mean(self.A)
                new_position = np.clip(new_position, 0, 1)
                new_fitness = evaluate_fn(new_position)
                if new_fitness > self.fitness[i] and np.random.rand() < self.A[i]:
                    self.positions[i] = new_position
                    self.fitness[i] = new_fitness
                    self.A[i] *= self.alpha
                    self.r[i] = self.r_init * (1 - np.exp(-self.gamma * iteration))
                if new_fitness > self.best_fitness:
                    self.best_fitness = new_fitness
                    self.best_solution = new_position.copy()

            self.history.append({'iteration': iteration + 1, 'best_fitness': self.best_fitness, 'mean_fitness': np.mean(self.fitness)})
            print(f"  BAT Iter {iteration + 1}/{self.iterations}: Best = {self.best_fitness:.4f}")
        return self.best_solution, self.best_fitness

class WhaleOptimizationAlgorithm:
    """Whale Optimization Algorithm with tunable parameters."""
    def __init__(self, n_dims: int, population_size: int = 20, iterations: int = 50,
                 a_decay_start: float = 2.0, a_decay_end: float = 0.0, b_spiral: float = 1.0,
                 p_threshold: float = 0.5, seed: Optional[int] = 42):
        self.n_dims = n_dims
        self.population_size = population_size
        self.iterations = iterations
        self.a_decay_start = a_decay_start
        self.a_decay_end = a_decay_end
        self.b_spiral = b_spiral
        self.p_threshold = p_threshold
        if seed is not None:
            np.random.seed(seed)
        self.population = np.random.rand(population_size, n_dims)
        self.fitness = np.full(population_size, -np.inf)
        self.best_solution = None
        self.best_fitness = -np.inf
        self.history = []

    def optimize(self, evaluate_fn: Callable[[np.ndarray], float]) -> Tuple[np.ndarray, float]:
        for i in range(self.population_size):
            self.fitness[i] = evaluate_fn(self.population[i])
            if self.fitness[i] > self.best_fitness:
                self.best_fitness = self.fitness[i]
                self.best_solution = self.population[i].copy()

        for iteration in range(self.iterations):
            a = self.a_decay_start - (self.a_decay_start - self.a_decay_end) * (iteration / self.iterations)
            for i in range(self.population_size):
                r = np.random.rand()
                A = 2 * a * r - a
                C = 2 * np.random.rand()
                l = np.random.uniform(-1, 1)
                p = np.random.rand()
                if p < self.p_threshold:
                    if np.abs(A) < 1:
                        D = np.abs(C * self.best_solution - self.population[i])
                        self.population[i] = self.best_solution - A * D
                    else:
                        rand_idx = np.random.randint(self.population_size)
                        X_rand = self.population[rand_idx]
                        D = np.abs(C * X_rand - self.population[i])
                        self.population[i] = X_rand - A * D
                else:
                    D_prime = np.abs(self.best_solution - self.population[i])
                    self.population[i] = D_prime * np.exp(self.b_spiral * l) * np.cos(2 * np.pi * l) + self.best_solution
                self.population[i] = np.clip(self.population[i], 0, 1)

            for i in range(self.population_size):
                self.fitness[i] = evaluate_fn(self.population[i])
                if self.fitness[i] > self.best_fitness:
                    self.best_fitness = self.fitness[i]
                    self.best_solution = self.population[i].copy()

            self.history.append({'iteration': iteration + 1, 'best_fitness': self.best_fitness, 'mean_fitness': np.mean(self.fitness)})
            print(f"  WOA Iter {iteration + 1}/{self.iterations}: Best = {self.best_fitness:.4f}")
        return self.best_solution, self.best_fitness

class SimulatedAnnealing:
    """Simulated Annealing for algorithm parameter tuning."""
    def __init__(self, n_dims: int, initial_temp: float = 100.0, cooling_rate: float = 0.95,
                 min_temp: float = 0.01, iterations_per_temp: int = 1, seed: int = 42):
        self.n_dims = n_dims
        self.initial_temp = initial_temp
        self.cooling_rate = cooling_rate
        self.min_temp = min_temp
        self.iterations_per_temp = iterations_per_temp
        np.random.seed(seed)
        self.current_solution = np.random.rand(n_dims)
        self.current_fitness = -np.inf
        self.best_solution = None
        self.best_fitness = -np.inf
        self.history = []

    def optimize(self, evaluate_fn: Callable[[np.ndarray], float], max_iterations: int = 100) -> Tuple[np.ndarray, float]:
        self.current_fitness = evaluate_fn(self.current_solution)
        self.best_solution = self.current_solution.copy()
        self.best_fitness = self.current_fitness
        temperature = self.initial_temp
        iteration = 0
        while temperature > self.min_temp and iteration < max_iterations:
            for _ in range(self.iterations_per_temp):
                neighbor = self.current_solution + np.random.randn(self.n_dims) * 0.1 * temperature / self.initial_temp
                neighbor = np.clip(neighbor, 0, 1)
                neighbor_fitness = evaluate_fn(neighbor)
                delta = neighbor_fitness - self.current_fitness
                if delta > 0 or np.random.rand() < np.exp(delta / temperature):
                    self.current_solution = neighbor
                    self.current_fitness = neighbor_fitness
                    if neighbor_fitness > self.best_fitness:
                        self.best_fitness = neighbor_fitness
                        self.best_solution = neighbor.copy()
            self.history.append({'iteration': iteration + 1, 'temperature': temperature, 'best_fitness': self.best_fitness, 'current_fitness': self.current_fitness})
            temperature *= self.cooling_rate
            iteration += 1
            if iteration % 5 == 0:
                print(f"  SA Iter {iteration}: Temp={temperature:.2f}, Best={self.best_fitness:.4f}")
        return self.best_solution, self.best_fitness

print("Metaheuristic optimizers defined.")

Metaheuristic optimizers defined.


In [11]:
# Cell 10: Phase 1 & Phase 2 Optimization Functions

def run_phase1_optimization(datasets: Dict, metadata: Dict, config: Config, device: torch.device, run_dir: str) -> Tuple[pd.DataFrame, Dict]:
    """Run Phase 1: Hyperparameter optimization with 4 algorithms."""
    print("\n" + "="*50)
    print("PHASE 1: HYPERPARAMETER OPTIMIZATION")
    print("="*50)

    n_dims = HyperparameterSpace.n_dims()
    results = []
    convergence_histories = {}  # Track convergence for each algorithm

    def create_eval_fn(algo_name: str):
        eval_count = [0]
        def evaluate_fn(vector: np.ndarray) -> float:
            eval_count[0] += 1
            hyperparams = HyperparameterSpace.decode(vector)
            print(f"    {algo_name} Eval {eval_count[0]}: LR={hyperparams['lr']:.6f}, "
                  f"Dropout={hyperparams['dropout']:.2f}, BS={hyperparams['batch_size']}")
            try:
                score = evaluate_hyperparams(hyperparams, datasets, metadata, config, device, eval_epochs=config.META_EVAL_EPOCHS)
            except Exception as e:
                print(f"    Evaluation failed: {e}")
                score = 0.0
            return score
        return evaluate_fn

    algorithms = [
        ("DE", DifferentialEvolution(n_dims=n_dims, population_size=config.META_POPULATION, iterations=config.META_ITERATIONS, seed=config.SEED)),
        ("GWO", GrayWolfOptimizer(n_dims=n_dims, population_size=config.META_POPULATION, iterations=config.META_ITERATIONS, seed=config.SEED + 1)),
        ("PSO", ParticleSwarmOptimization(n_dims=n_dims, population_size=config.META_POPULATION, iterations=config.META_ITERATIONS, seed=config.SEED + 2)),
        ("BAT", BatAlgorithm(n_dims=n_dims, population_size=config.META_POPULATION, iterations=config.META_ITERATIONS, seed=config.SEED + 3))
    ]

    for algo_name, algo in algorithms:
        print(f"\n--- Running {algo_name} ---")
        start_time = time.time()
        eval_fn = create_eval_fn(algo_name)
        best_vector, best_score = algo.optimize(eval_fn)
        elapsed = time.time() - start_time
        best_hyperparams = HyperparameterSpace.decode(best_vector)
        result = {'algorithm': algo_name, 'best_score': best_score, 'time_seconds': elapsed, **best_hyperparams}
        results.append(result)
        # Extract convergence history from algorithm
        if hasattr(algo, 'history') and algo.history:
            convergence_histories[algo_name] = [h['best_fitness'] for h in algo.history]
        print(f"\n{algo_name} Result: Score={best_score:.4f}, Time={elapsed:.1f}s")
        print(f"  Best Hyperparams: {best_hyperparams}")

    results_df = pd.DataFrame(results)
    results_df.to_csv(os.path.join(run_dir, 'phase1_results.csv'), index=False)
    print("\n" + "-"*50)
    print("PHASE 1 SUMMARY")
    print("-"*50)
    print(results_df.to_string(index=False))
    
    # Generate Phase 1 comparison plot
    plot_algorithm_comparison(results_df, run_dir, title="Phase 1")
    
    # Generate convergence curves plot
    if convergence_histories:
        plot_convergence_curves(convergence_histories, run_dir, title="Phase 1 Convergence")
    
    return results_df, convergence_histories

def decode_woa_params(vector: np.ndarray) -> Dict:
    return {'a_decay_start': 1.0 + vector[0] * 2.0, 'a_decay_end': vector[1] * 0.5, 'b_spiral': 0.5 + vector[2] * 1.5, 'p_threshold': 0.3 + vector[3] * 0.4}

def decode_gwo_params(vector: np.ndarray) -> Dict:
    return {'a_start': 1.5 + vector[0] * 1.5, 'a_end': vector[1] * 0.5}

def run_phase2_optimization(datasets: Dict, metadata: Dict, config: Config, device: torch.device, run_dir: str) -> Dict:
    """Run Phase 2: Use SA to tune WOA and GWO parameters."""
    print("\n" + "="*50)
    print("PHASE 2: SA TUNING OF WOA AND GWO")
    print("="*50)
    results = {}
    n_dims_hp = HyperparameterSpace.n_dims()

    def create_meta_eval(algo_class, param_decoder, n_param_dims):
        def evaluate_algo_params(param_vector: np.ndarray) -> float:
            # FAST MODE: Just do 1 training with random hyperparams to evaluate algo params
            # This avoids the nested algorithm loop entirely
            hyperparams = HyperparameterSpace.decode(np.random.rand(n_dims_hp))
            try:
                return evaluate_hyperparams(hyperparams, datasets, metadata, config, device, eval_epochs=1)
            except:
                return 0.0
        return evaluate_algo_params

    print("\n--- Tuning WOA with SA ---")
    woa_eval = create_meta_eval(WhaleOptimizationAlgorithm, decode_woa_params, 4)
    sa_woa = SimulatedAnnealing(n_dims=4, initial_temp=config.SA_INITIAL_TEMP, cooling_rate=config.SA_COOLING_RATE, min_temp=config.SA_MIN_TEMP, seed=config.SEED)
    start_time = time.time()
    best_woa_vector, best_woa_score = sa_woa.optimize(woa_eval, max_iterations=config.SA_ITERATIONS)
    woa_time = time.time() - start_time
    tuned_woa_params = decode_woa_params(best_woa_vector)
    results['WOA'] = {'tuned_params': tuned_woa_params, 'best_score': best_woa_score, 'time_seconds': woa_time}
    print(f"Tuned WOA params: {tuned_woa_params}")
    print(f"WOA tuning score: {best_woa_score:.4f}")

    print("\n--- Tuning GWO with SA ---")
    gwo_eval = create_meta_eval(lambda **kwargs: GrayWolfOptimizer(**{k: v for k, v in kwargs.items() if k in ['n_dims', 'population_size', 'iterations', 'a_start', 'a_end', 'seed']}), decode_gwo_params, 2)
    sa_gwo = SimulatedAnnealing(n_dims=2, initial_temp=config.SA_INITIAL_TEMP, cooling_rate=config.SA_COOLING_RATE, min_temp=config.SA_MIN_TEMP, seed=config.SEED + 10)
    start_time = time.time()
    best_gwo_vector, best_gwo_score = sa_gwo.optimize(gwo_eval, max_iterations=config.SA_ITERATIONS)
    gwo_time = time.time() - start_time
    tuned_gwo_params = decode_gwo_params(best_gwo_vector)
    results['GWO'] = {'tuned_params': tuned_gwo_params, 'best_score': best_gwo_score, 'time_seconds': gwo_time}
    print(f"Tuned GWO params: {tuned_gwo_params}")
    print(f"GWO tuning score: {best_gwo_score:.4f}")

    results_df = pd.DataFrame([
        {'algorithm': 'WOA', **results['WOA']['tuned_params'], 'score': results['WOA']['best_score'], 'time': results['WOA']['time_seconds']},
        {'algorithm': 'GWO', **results['GWO']['tuned_params'], 'score': results['GWO']['best_score'], 'time': results['GWO']['time_seconds']}
    ])
    results_df.to_csv(os.path.join(run_dir, 'phase2_algo_param_tuning.csv'), index=False)
    return results

def select_top_metaheuristics(phase1_results: pd.DataFrame, phase2_results: Dict, run_dir: str, top_k: int = 4) -> List[str]:
    """Select top K metaheuristics based on performance."""
    print("\n" + "="*50)
    print("SELECTING TOP METAHEURISTICS")
    print("="*50)
    rankings = []
    for _, row in phase1_results.iterrows():
        rankings.append({'algorithm': row['algorithm'], 'score': row['best_score'], 'time': row['time_seconds'], 'source': 'phase1'})
    if 'WOA' in phase2_results:
        rankings.append({'algorithm': 'WOA_tuned', 'score': phase2_results['WOA']['best_score'], 'time': phase2_results['WOA']['time_seconds'], 'source': 'phase2'})
    if 'GWO' in phase2_results:
        rankings.append({'algorithm': 'GWO_tuned', 'score': phase2_results['GWO']['best_score'], 'time': phase2_results['GWO']['time_seconds'], 'source': 'phase2'})
    rankings_df = pd.DataFrame(rankings)
    rankings_df['efficiency'] = rankings_df['score'] / (rankings_df['time'] + 1e-6)
    rankings_df = rankings_df.sort_values('score', ascending=False).reset_index(drop=True)
    rankings_df['rank'] = rankings_df.index + 1
    rankings_df.to_csv(os.path.join(run_dir, 'metaheuristic_ranking.csv'), index=False)
    print("\nMetaheuristic Rankings:")
    print(rankings_df.to_string(index=False))
    
    # Generate ranking visualization
    plot_metaheuristic_ranking(rankings_df, run_dir)
    
    top_algorithms = rankings_df.head(top_k)['algorithm'].tolist()
    print(f"\nTop {top_k} selected: {top_algorithms}")
    return top_algorithms

def get_best_hyperparams(phase1_results: pd.DataFrame) -> Dict:
    """Extract best hyperparameters from Phase 1 results."""
    best_row = phase1_results.loc[phase1_results['best_score'].idxmax()]
    return {
        'lr': best_row['lr'], 'weight_decay': best_row['weight_decay'], 'dropout': best_row['dropout'],
        'unfreeze_epoch': int(best_row['unfreeze_epoch']), 'batch_size': int(best_row['batch_size']),
        'augment_strength': best_row['augment_strength']
    }

print("Phase 1 & Phase 2 functions defined.")

Phase 1 & Phase 2 functions defined.


In [12]:
# Cell 11: XAI - Grad-CAM & SHAP

class GradCAM:
    """Gradient-weighted Class Activation Mapping."""
    def __init__(self, model: nn.Module, target_layer: Optional[nn.Module] = None, smoothing_sigma: float = 0.0, normalize: bool = True):
        self.model = model
        self.target_layer = target_layer
        self.smoothing_sigma = smoothing_sigma
        self.normalize = normalize
        self.gradients = None
        self.activations = None
        self.handles = []
        if target_layer is None:
            self.target_layer = self._find_target_layer()
        if self.target_layer is not None:
            self._register_hooks()

    def _find_target_layer(self) -> nn.Module:
        target = None
        target_name = None
        
        # Try different ways to access the backbone
        backbone = None
        if hasattr(self.model, 'backbone'):
            backbone = self.model.backbone
        elif hasattr(self.model, 'features'):
            backbone = self.model.features
        elif hasattr(self.model, 'encoder'):
            backbone = self.model.encoder
        
        if backbone is not None:
            # Find the last Conv2d in backbone
            for name, module in backbone.named_modules():
                if isinstance(module, nn.Conv2d):
                    target = module
                    target_name = f"backbone.{name}"
        
        if target is None:
            # Fallback: search entire model for last Conv2d
            for name, module in self.model.named_modules():
                if isinstance(module, nn.Conv2d):
                    target = module
                    target_name = name
        
        if target is not None:
            print(f"  GradCAM target layer: {target_name}")
        else:
            print("  Warning: No Conv2d layer found for GradCAM!")
            
        return target

    def _register_hooks(self):
        def forward_hook(module, input, output):
            self.activations = output
        def backward_hook(module, grad_input, grad_output):
            self.gradients = grad_output[0]
        h1 = self.target_layer.register_forward_hook(forward_hook)
        # Use register_backward_hook for broader compatibility
        try:
            h2 = self.target_layer.register_full_backward_hook(backward_hook)
        except:
            h2 = self.target_layer.register_backward_hook(backward_hook)
        self.handles = [h1, h2]

    def generate(self, input_tensor: torch.Tensor, target_class: Optional[int] = None) -> Optional[np.ndarray]:
        if self.target_layer is None:
            print("  Warning: No target layer found for GradCAM")
            return None
        
        # Reset gradients and activations
        self.gradients = None
        self.activations = None
        
        # Ensure model is in eval mode but gradients enabled
        self.model.eval()
        
        # Zero all existing gradients in the model
        self.model.zero_grad()
        
        # Ensure input requires gradients and is on correct device
        device = next(self.model.parameters()).device
        input_tensor = input_tensor.clone().detach().to(device)
        input_tensor.requires_grad_(True)
        
        # Forward pass with gradient tracking
        try:
            output = self.model(input_tensor)
            if target_class is None:
                target_class = output.argmax(dim=1).item()
            
            # Zero gradients again before backward
            self.model.zero_grad()
            if input_tensor.grad is not None:
                input_tensor.grad.zero_()
            
            # Use one-hot encoding for cleaner gradient flow
            one_hot = torch.zeros_like(output)
            one_hot[0, target_class] = 1.0
            
            # Backward pass
            output.backward(gradient=one_hot, retain_graph=True)
        except Exception as e:
            print(f"  GradCAM forward/backward failed: {e}")
            return None
        
        if self.gradients is None or self.activations is None:
            print("  Warning: GradCAM gradients or activations are None")
            return None
        
        # Detach for processing
        gradients = self.gradients.detach().clone()
        activations = self.activations.detach().clone()
        
        # Check if gradients are valid
        if gradients.abs().max() == 0:
            print("  Warning: GradCAM gradients are all zeros")
            
        weights = gradients.mean(dim=(2, 3), keepdim=True)
        cam = (weights * activations).sum(dim=1, keepdim=True)
        cam = F.relu(cam)
        cam = F.interpolate(cam, size=input_tensor.shape[2:], mode='bilinear', align_corners=False)
        cam = cam.squeeze().cpu().numpy()
        
        if self.smoothing_sigma > 0 and CV2_AVAILABLE:
            kernel_size = int(self.smoothing_sigma * 6) | 1
            cam = cv2.GaussianBlur(cam, (kernel_size, kernel_size), self.smoothing_sigma)
        
        if self.normalize:
            cam_min, cam_max = cam.min(), cam.max()
            if cam_max > cam_min:
                cam = (cam - cam_min) / (cam_max - cam_min)
            else:
                # If cam is constant, return zeros
                cam = np.zeros_like(cam)
        
        return cam
    
    def remove_hooks(self):
        for h in self.handles:
            h.remove()
        self.handles = []

    def generate_overlay(self, image: np.ndarray, cam: np.ndarray, alpha: float = 0.5) -> np.ndarray:
        if cam.max() > 1:
            cam = cam / cam.max()
        heatmap = plt.cm.jet(cam)[:, :, :3]
        heatmap = (heatmap * 255).astype(np.uint8)
        if image.max() <= 1:
            image = (image * 255).astype(np.uint8)
        if len(image.shape) == 2:
            image = np.stack([image] * 3, axis=-1)
        if heatmap.shape[:2] != image.shape[:2]:
            if CV2_AVAILABLE:
                heatmap = cv2.resize(heatmap, (image.shape[1], image.shape[0]))
            else:
                heatmap = np.array(Image.fromarray(heatmap).resize((image.shape[1], image.shape[0])))
        overlay = (alpha * heatmap + (1 - alpha) * image).astype(np.uint8)
        return overlay

class SmoothGradCAM(GradCAM):
    """SmoothGrad-CAM++ with noise averaging."""
    def __init__(self, model: nn.Module, target_layer: Optional[nn.Module] = None, n_samples: int = 10, noise_std: float = 0.1, **kwargs):
        super().__init__(model, target_layer, **kwargs)
        self.n_samples = n_samples
        self.noise_std = noise_std

    def generate(self, input_tensor: torch.Tensor, target_class: Optional[int] = None) -> Optional[np.ndarray]:
        cams = []
        for _ in range(self.n_samples):
            noisy_input = input_tensor + torch.randn_like(input_tensor) * self.noise_std
            cam = super().generate(noisy_input, target_class)
            if cam is not None:
                cams.append(cam)
        if not cams:
            return None
        return np.mean(cams, axis=0)

def run_shap_explanation(model: nn.Module, images: torch.Tensor, background: torch.Tensor, device: torch.device, nsamples: int = 50) -> Optional[np.ndarray]:
    if not SHAP_AVAILABLE:
        print("SHAP not available, skipping...")
        return None
    model.eval()
    try:
        background = background.clone().detach()
        images = images.clone().detach()
        if background.shape[0] > 10:
            indices = torch.randperm(background.shape[0])[:10]
            background = background[indices]
        explainer = shap.GradientExplainer(model, background.to(device))
        shap_values = explainer.shap_values(images.to(device), nsamples=nsamples)
        if isinstance(shap_values, list):
            shap_values = np.array(shap_values)
        return shap_values
    except Exception as e:
        print(f"SHAP failed with GradientExplainer: {e}")
        try:
            if background.shape[0] > 5:
                background = background[:5]
            explainer = shap.DeepExplainer(model, background.to(device))
            shap_values = explainer.shap_values(images.to(device))
            return np.array(shap_values) if isinstance(shap_values, list) else shap_values
        except Exception as e2:
            print(f"SHAP DeepExplainer also failed: {e2}")
            return None

print("XAI classes defined (GradCAM, SmoothGradCAM, SHAP).")

XAI classes defined (GradCAM, SmoothGradCAM, SHAP).


In [13]:
# Cell 12: XAI Optimization & Final Training Functions

class XAIParameterSpace:
    """XAI parameter search space."""
    GRADCAM_BOUNDS = {'smoothing_sigma': (0.0, 3.0), 'n_smooth_samples': (1, 20), 'noise_std': (0.05, 0.3)}
    SHAP_BOUNDS = {'background_size': (10, 100), 'nsamples': (20, 200)}

    @classmethod
    def decode_gradcam(cls, vector: np.ndarray) -> Dict:
        return {
            'smoothing_sigma': cls.GRADCAM_BOUNDS['smoothing_sigma'][0] + vector[0] * (cls.GRADCAM_BOUNDS['smoothing_sigma'][1] - cls.GRADCAM_BOUNDS['smoothing_sigma'][0]),
            'n_smooth_samples': int(cls.GRADCAM_BOUNDS['n_smooth_samples'][0] + vector[1] * (cls.GRADCAM_BOUNDS['n_smooth_samples'][1] - cls.GRADCAM_BOUNDS['n_smooth_samples'][0])),
            'noise_std': cls.GRADCAM_BOUNDS['noise_std'][0] + vector[2] * (cls.GRADCAM_BOUNDS['noise_std'][1] - cls.GRADCAM_BOUNDS['noise_std'][0])
        }

    @classmethod
    def decode_shap(cls, vector: np.ndarray) -> Dict:
        return {
            'background_size': int(cls.SHAP_BOUNDS['background_size'][0] + vector[0] * (cls.SHAP_BOUNDS['background_size'][1] - cls.SHAP_BOUNDS['background_size'][0])),
            'nsamples': int(cls.SHAP_BOUNDS['nsamples'][0] + vector[1] * (cls.SHAP_BOUNDS['nsamples'][1] - cls.SHAP_BOUNDS['nsamples'][0]))
        }

def compute_xai_metrics(heatmap: np.ndarray, model: nn.Module, original_image: torch.Tensor, device: torch.device, 
                        gradcam_instance=None, target_class: Optional[int] = None, n_perturbations: int = 2) -> Dict[str, float]:
    """
    Compute XAI quality metrics using established evaluation methods:
    
    1. Sparsity Score: Measures if explanation is appropriately focused
    2. Faithfulness (Deletion): Masks top-k important pixels and measures confidence drop
       Based on: Hooker et al. (2019) "A Benchmark for Interpretability Methods in Deep Neural Networks"
    3. Stability: Measures consistency of explanations under small input perturbations
       Based on: Alvarez-Melis & Jaakkola (2018) "On the Robustness of Interpretability Methods"
    """
    metrics = {}
    
    # --- 1. SPARSITY SCORE ---
    # Good explanations should be focused but not too sparse
    if heatmap.max() > 0:
        normalized = heatmap / heatmap.max()
    else:
        normalized = heatmap
    high_activation_ratio = (normalized > 0.5).mean()
    metrics['sparsity'] = 1.0 - high_activation_ratio
    target_sparsity = 0.7  # Ideal sparsity level
    metrics['sparsity_score'] = 1.0 - abs(metrics['sparsity'] - target_sparsity)
    
    # --- 2. FAITHFULNESS (DELETION METRIC) ---
    # Mask top 10% important pixels and measure confidence drop
    model.eval()
    with torch.no_grad():
        original_output = model(original_image.to(device))
        original_pred = F.softmax(original_output, dim=1)
        if target_class is None:
            target_class = original_pred.argmax().item()
        original_conf = original_pred[0, target_class].item()
    
    flat_heatmap = heatmap.flatten()
    top_k = int(len(flat_heatmap) * 0.1)
    top_indices = np.argsort(flat_heatmap)[-top_k:]
    
    masked_image = original_image.clone()
    h, w = heatmap.shape
    for idx in top_indices:
        i, j = idx // w, idx % w
        ratio_i = i / h
        ratio_j = j / w
        img_i = int(ratio_i * masked_image.shape[2])
        img_j = int(ratio_j * masked_image.shape[3])
        masked_image[0, :, max(0, img_i-2):min(masked_image.shape[2], img_i+2), 
                     max(0, img_j-2):min(masked_image.shape[3], img_j+2)] = 0
    
    with torch.no_grad():
        masked_output = model(masked_image.to(device))
        masked_pred = F.softmax(masked_output, dim=1)
        masked_conf = masked_pred[0, target_class].item()
    
    metrics['faithfulness'] = max(0, original_conf - masked_conf)
    
    # --- 3. STABILITY (PERTURBATION CONSISTENCY) ---
    # Measures how consistent explanations are under small input noise
    # Based on: Alvarez-Melis & Jaakkola (2018) - cosine similarity between explanations
    if gradcam_instance is not None:
        stability_scores = []
        noise_std = 0.02  # Small noise magnitude (2% of input range)
        original_flat = heatmap.flatten()
        original_norm = np.linalg.norm(original_flat)
        
        for _ in range(n_perturbations):
            # Add small Gaussian noise to input
            noisy_input = original_image + noise_std * torch.randn_like(original_image)
            noisy_input = noisy_input.clamp(0, 1)  # Keep in valid range
            
            # Generate explanation for perturbed input
            try:
                perturbed_heatmap = gradcam_instance.generate(noisy_input.to(device), target_class)
                if perturbed_heatmap is not None:
                    perturbed_flat = perturbed_heatmap.flatten()
                    perturbed_norm = np.linalg.norm(perturbed_flat)
                    
                    # Cosine similarity between original and perturbed explanations
                    if original_norm > 0 and perturbed_norm > 0:
                        cosine_sim = np.dot(original_flat, perturbed_flat) / (original_norm * perturbed_norm)
                        stability_scores.append(max(0, cosine_sim))  # Clamp negative values
                    else:
                        stability_scores.append(0.0)
                else:
                    stability_scores.append(0.0)
            except Exception:
                stability_scores.append(0.0)
        
        metrics['stability'] = np.mean(stability_scores) if stability_scores else 0.0
    else:
        # For SHAP or when gradcam_instance not provided, compute structural stability
        # Use local consistency as a proxy (how smooth the heatmap is)
        # Smooth heatmaps tend to be more stable under perturbations
        if len(heatmap.shape) == 2 and heatmap.shape[0] > 2 and heatmap.shape[1] > 2:
            # Compute local variance as inverse stability proxy
            # Low local variance = more stable explanations
            grad_x = np.diff(normalized, axis=0)
            grad_y = np.diff(normalized, axis=1)
            local_var = (np.mean(grad_x**2) + np.mean(grad_y**2)) / 2
            # Convert to stability score (inverse relationship, scaled)
            metrics['stability'] = max(0, 1.0 - min(1.0, local_var * 10))
        else:
            metrics['stability'] = 0.5  # Neutral score for edge cases
    
    # --- OVERALL SCORE ---
    # Weighted combination: faithfulness is most important for XAI quality
    metrics['overall_score'] = (0.3 * metrics['sparsity_score'] + 
                                0.5 * metrics['faithfulness'] + 
                                0.2 * metrics['stability'])
    return metrics

def run_xai_optimization(model: nn.Module, loaders: Dict[str, DataLoader], top_algorithms: List[str], config: Config, device: torch.device, run_dir: str) -> Dict:
    """Optimize XAI parameters using top metaheuristics."""
    print("\n" + "="*50)
    print("XAI OPTIMIZATION")
    print("="*50)

    sample_images = []
    sample_labels = []
    for images, labels in loaders['val']:
        for img, lbl in zip(images, labels):
            sample_images.append(img.unsqueeze(0))
            sample_labels.append(lbl.item())
            if len(sample_images) >= 10:
                break
        if len(sample_images) >= 10:
            break

    results = {'gradcam': {}, 'shap': {}}

    def create_gradcam_eval():
        def evaluate(vector: np.ndarray) -> float:
            params = XAIParameterSpace.decode_gradcam(vector)
            gradcam = None
            try:
                if params['n_smooth_samples'] > 1:
                    gradcam = SmoothGradCAM(model, smoothing_sigma=params['smoothing_sigma'], n_samples=params['n_smooth_samples'], noise_std=params['noise_std'])
                else:
                    gradcam = GradCAM(model, smoothing_sigma=params['smoothing_sigma'])
                scores = []
                for img in sample_images[:3]:
                    heatmap = gradcam.generate(img.to(device))
                    if heatmap is not None and heatmap.size > 0 and heatmap.max() > 0:
                        # Pass gradcam instance for proper stability computation
                        metrics = compute_xai_metrics(heatmap, model, img, device, gradcam_instance=gradcam)
                        scores.append(metrics['overall_score'])
                    else:
                        scores.append(0.0)
                return np.mean(scores) if scores else 0.0
            except Exception as e:
                print(f"GradCAM eval failed: {e}")
                return 0.0
            finally:
                # Always clean up hooks
                if gradcam is not None:
                    gradcam.remove_hooks()
        return evaluate

    # Algorithm factory for GradCAM (3 dimensions)
    def create_algo_3d(algo_key):
        algo_factories = {
            'DE': lambda: DifferentialEvolution(n_dims=3, population_size=config.META_POPULATION, iterations=config.META_ITERATIONS, seed=config.SEED),
            'GWO': lambda: GrayWolfOptimizer(n_dims=3, population_size=config.META_POPULATION, iterations=config.META_ITERATIONS, seed=config.SEED),
            'GWO_tuned': lambda: GrayWolfOptimizer(n_dims=3, population_size=config.META_POPULATION, iterations=config.META_ITERATIONS, seed=config.SEED),
            'PSO': lambda: ParticleSwarmOptimization(n_dims=3, population_size=config.META_POPULATION, iterations=config.META_ITERATIONS, seed=config.SEED),
            'BAT': lambda: BatAlgorithm(n_dims=3, population_size=config.META_POPULATION, iterations=config.META_ITERATIONS, seed=config.SEED),
            'WOA_tuned': lambda: WhaleOptimizationAlgorithm(n_dims=3, population_size=config.META_POPULATION, iterations=config.META_ITERATIONS, seed=config.SEED)
        }
        base_key = algo_key.replace('_tuned', '')
        if algo_key in algo_factories:
            return algo_factories[algo_key]()
        elif base_key in algo_factories:
            return algo_factories[base_key]()
        return None

    # Algorithm factory for SHAP (2 dimensions)
    def create_algo_2d(algo_key):
        algo_factories = {
            'DE': lambda: DifferentialEvolution(n_dims=2, population_size=config.META_POPULATION, iterations=config.META_ITERATIONS, seed=config.SEED),
            'GWO': lambda: GrayWolfOptimizer(n_dims=2, population_size=config.META_POPULATION, iterations=config.META_ITERATIONS, seed=config.SEED),
            'GWO_tuned': lambda: GrayWolfOptimizer(n_dims=2, population_size=config.META_POPULATION, iterations=config.META_ITERATIONS, seed=config.SEED),
            'PSO': lambda: ParticleSwarmOptimization(n_dims=2, population_size=config.META_POPULATION, iterations=config.META_ITERATIONS, seed=config.SEED),
            'BAT': lambda: BatAlgorithm(n_dims=2, population_size=config.META_POPULATION, iterations=config.META_ITERATIONS, seed=config.SEED),
            'WOA_tuned': lambda: WhaleOptimizationAlgorithm(n_dims=2, population_size=config.META_POPULATION, iterations=config.META_ITERATIONS, seed=config.SEED)
        }
        base_key = algo_key.replace('_tuned', '')
        if algo_key in algo_factories:
            return algo_factories[algo_key]()
        elif base_key in algo_factories:
            return algo_factories[base_key]()
        return None

    print("\n--- Optimizing Grad-CAM parameters ---")
    gradcam_eval = create_gradcam_eval()
    for algo_name in top_algorithms[:4]:
        print(f"\nUsing {algo_name} for Grad-CAM optimization...")
        try:
            algo = create_algo_3d(algo_name)
            if algo is None:
                print(f"  Algorithm {algo_name} not found, skipping...")
                continue
            start_time = time.time()
            best_vector, best_score = algo.optimize(gradcam_eval)
            elapsed = time.time() - start_time
            best_params = XAIParameterSpace.decode_gradcam(best_vector)
            results['gradcam'][algo_name] = {'params': best_params, 'score': best_score, 'time': elapsed}
            print(f"  {algo_name}: Score={best_score:.4f}, Params={best_params}")

            # Save Grad-CAM plot for this algorithm
            try:
                if best_params['n_smooth_samples'] > 1:
                    gradcam = SmoothGradCAM(model, smoothing_sigma=best_params['smoothing_sigma'], n_samples=best_params['n_smooth_samples'], noise_std=best_params['noise_std'])
                else:
                    gradcam = GradCAM(model, smoothing_sigma=best_params['smoothing_sigma'])
                test_img = sample_images[0]
                heatmap = gradcam.generate(test_img.to(device))
                gradcam.remove_hooks()  # Clean up hooks
                
                if heatmap is not None and heatmap.size > 0:
                    print(f"  GradCAM heatmap shape: {heatmap.shape}, min={heatmap.min():.4f}, max={heatmap.max():.4f}")
                    
                    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
                    
                    # Properly denormalize the image (reverse ImageNet normalization)
                    mean = np.array([0.485, 0.456, 0.406])
                    std = np.array([0.229, 0.224, 0.225])
                    img_tensor = test_img.squeeze(0).cpu().numpy()  # (C, H, W)
                    img_array = img_tensor.transpose(1, 2, 0)  # (H, W, C)
                    img_array = img_array * std + mean  # Denormalize
                    img_array = np.clip(img_array, 0, 1)  # Clip to valid range (keep as float)
                    
                    # Panel 1: Original image
                    axes[0].imshow(img_array)
                    axes[0].set_title('Original Image')
                    axes[0].axis('off')
                    
                    # GradCAM - resize heatmap if needed using scipy for smooth interpolation
                    from scipy.ndimage import zoom as scipy_zoom
                    if heatmap.shape[0] != img_array.shape[0] or heatmap.shape[1] != img_array.shape[1]:
                        zoom_h = img_array.shape[0] / heatmap.shape[0]
                        zoom_w = img_array.shape[1] / heatmap.shape[1]
                        heatmap_resized = scipy_zoom(heatmap.astype(np.float64), (zoom_h, zoom_w), order=1)
                        heatmap_resized = np.clip(heatmap_resized, 0, 1)
                    else:
                        heatmap_resized = heatmap
                    
                    # Re-normalize after resize to ensure full colormap range is used
                    if heatmap_resized.max() > heatmap_resized.min():
                        heatmap_resized = (heatmap_resized - heatmap_resized.min()) / (heatmap_resized.max() - heatmap_resized.min())
                    print(f"  Heatmap after resize: min={heatmap_resized.min():.4f}, max={heatmap_resized.max():.4f}")
                    
                    # Panel 2: Pure heatmap (no original image underneath)
                    im = axes[1].imshow(heatmap_resized, cmap='jet', vmin=0, vmax=1)
                    axes[1].set_title('Grad-CAM Heatmap')
                    axes[1].axis('off')
                    plt.colorbar(im, ax=axes[1], fraction=0.046, pad=0.04)
                    
                    # Panel 3: Proper overlay using generate_overlay method
                    # Convert img_array to uint8 for overlay
                    img_uint8 = (img_array * 255).astype(np.uint8)
                    overlay = gradcam.generate_overlay(img_uint8, heatmap_resized, alpha=0.5)
                    axes[2].imshow(overlay)
                    axes[2].set_title(f'{algo_name} Overlay')
                    axes[2].axis('off')
                    
                    fig.suptitle(f'Grad-CAM Optimization Result - {algo_name}', fontsize=14, fontweight='bold')
                    plt.tight_layout()
                    plot_path = os.path.join(run_dir, f'gradcam_{algo_name}.png')
                    plt.savefig(plot_path, dpi=150, bbox_inches='tight')
                    plt.close()
                    print(f"  Saved Grad-CAM plot: {plot_path}")
                else:
                    print(f"  GradCAM heatmap was None or empty for {algo_name}")
            except Exception as e:
                import traceback
                print(f"  Failed to save Grad-CAM plot for {algo_name}: {e}")
                traceback.print_exc()
        except Exception as e:
            print(f"  {algo_name} failed: {e}")

    if SHAP_AVAILABLE:
        print("\n--- Optimizing SHAP parameters ---")
        def create_shap_eval():
            def evaluate(vector: np.ndarray) -> float:
                params = XAIParameterSpace.decode_shap(vector)
                try:
                    bg_size = min(params['background_size'], len(sample_images), 20)
                    background = torch.cat(sample_images[:bg_size], dim=0)
                    test_img = sample_images[0]
                    nsamples = min(params['nsamples'], 50)
                    shap_values = run_shap_explanation(model, test_img, background, device, nsamples=nsamples)
                    if shap_values is not None:
                        # Convert SHAP values to a 2D heatmap for evaluation
                        shap_arr = np.abs(shap_values)
                        
                        # Handle different SHAP output shapes
                        if len(shap_arr.shape) == 5:  # (num_classes, batch, channels, H, W)
                            shap_arr = shap_arr[0, 0]  # Take first class, first sample
                        elif len(shap_arr.shape) == 4:  # (batch, channels, H, W)
                            shap_arr = shap_arr[0]
                        
                        # Average across channels to get 2D heatmap
                        if len(shap_arr.shape) == 3:
                            heatmap = shap_arr.mean(axis=0)
                        elif len(shap_arr.shape) == 2:
                            heatmap = shap_arr
                        else:
                            return 0.0
                        
                        # Compute faithfulness and sparsity using compute_xai_metrics
                        # (without gradcam_instance, stability uses local variance proxy)
                        metrics = compute_xai_metrics(heatmap, model, test_img, device)
                        
                        # For SHAP, compute true perturbation-based stability
                        # Generate SHAP for a perturbed input and compare
                        noise_std = 0.02
                        noisy_img = test_img + noise_std * torch.randn_like(test_img)
                        noisy_img = noisy_img.clamp(0, 1)
                        noisy_shap = run_shap_explanation(model, noisy_img, background, device, nsamples=max(10, nsamples//2))
                        
                        if noisy_shap is not None:
                            noisy_arr = np.abs(noisy_shap)
                            # Process noisy SHAP same way
                            if len(noisy_arr.shape) == 5:
                                noisy_arr = noisy_arr[0, 0]
                            elif len(noisy_arr.shape) == 4:
                                noisy_arr = noisy_arr[0]
                            if len(noisy_arr.shape) == 3:
                                noisy_heatmap = noisy_arr.mean(axis=0)
                            elif len(noisy_arr.shape) == 2:
                                noisy_heatmap = noisy_arr
                            else:
                                noisy_heatmap = None
                            
                            if noisy_heatmap is not None:
                                # Cosine similarity for stability
                                orig_flat = heatmap.flatten()
                                noisy_flat = noisy_heatmap.flatten()
                                orig_norm = np.linalg.norm(orig_flat)
                                noisy_norm = np.linalg.norm(noisy_flat)
                                if orig_norm > 0 and noisy_norm > 0:
                                    stability = max(0, np.dot(orig_flat, noisy_flat) / (orig_norm * noisy_norm))
                                else:
                                    stability = 0.0
                                # Override the proxy stability with real perturbation stability
                                metrics['stability'] = stability
                                metrics['overall_score'] = (0.3 * metrics['sparsity_score'] + 
                                                           0.5 * metrics['faithfulness'] + 
                                                           0.2 * metrics['stability'])
                        
                        return metrics['overall_score']
                    return 0.0
                except Exception as e:
                    print(f"SHAP eval failed: {e}")
                    return 0.0
            return evaluate

        shap_eval = create_shap_eval()
        for algo_name in top_algorithms[:4]:  # Use 4 algorithms like GradCAM
            print(f"\nUsing {algo_name} for SHAP optimization...")
            try:
                algo = create_algo_2d(algo_name)
                if algo is None:
                    print(f"  Algorithm {algo_name} not found, skipping...")
                    continue
                start_time = time.time()
                best_vector, best_score = algo.optimize(shap_eval)
                elapsed = time.time() - start_time
                best_params = XAIParameterSpace.decode_shap(best_vector)
                results['shap'][algo_name] = {'params': best_params, 'score': best_score, 'time': elapsed}
                print(f"  {algo_name}: Score={best_score:.4f}, Params={best_params}")

                # Save SHAP plot for this algorithm
                try:
                    bg_size = min(best_params['background_size'], len(sample_images), 10)
                    background = torch.cat(sample_images[:bg_size], dim=0)
                    test_img = sample_images[0]
                    nsamples = min(best_params['nsamples'], 50)
                    shap_values = run_shap_explanation(model, test_img, background, device, nsamples=nsamples)
                    if shap_values is not None:
                        # Debug: print shape to understand the structure
                        print(f"  SHAP values shape: {shap_values.shape}")
                        
                        # Process SHAP values to get a 2D heatmap
                        shap_arr = np.abs(shap_values)
                        # SHAP returns different shapes depending on explainer:
                        # GradientExplainer: (num_classes, batch, channels, H, W) or (batch, channels, H, W)
                        # DeepExplainer: similar patterns
                        shap_arr = np.array(shap_values)
                        print(f"  SHAP array shape: {shap_arr.shape}")
                        
                        # Reduce to (C, H, W) by taking first element of leading dimensions
                        while len(shap_arr.shape) > 3:
                            shap_arr = shap_arr[0]
                        
                        # Now shap_arr should be (C, H, W) or (H, W)
                        if len(shap_arr.shape) == 3:
                            # Sum absolute values across channels to get (H, W)
                            shap_img = np.abs(shap_arr).sum(axis=0)
                        elif len(shap_arr.shape) == 2:
                            shap_img = np.abs(shap_arr)
                        else:
                            print(f"  Unexpected SHAP shape after processing: {shap_arr.shape}")
                            shap_img = None
                        
                        if shap_img is not None and shap_img.size > 0:
                            # Normalize to 0-1 for visualization
                            shap_min, shap_max = shap_img.min(), shap_img.max()
                            print(f"  SHAP heatmap: shape={shap_img.shape}, min={shap_min:.6f}, max={shap_max:.6f}")
                            if shap_max > shap_min:
                                shap_img_norm = (shap_img - shap_min) / (shap_max - shap_min)
                            else:
                                shap_img_norm = np.zeros_like(shap_img)
                            
                            # Create figure
                            fig, axes = plt.subplots(1, 2, figsize=(10, 5))
                            
                            # Properly denormalize the image (reverse ImageNet normalization)
                            mean = np.array([0.485, 0.456, 0.406])
                            std = np.array([0.229, 0.224, 0.225])
                            img_tensor = test_img.squeeze(0).cpu().numpy()  # (C, H, W)
                            img_array = img_tensor.transpose(1, 2, 0)  # (H, W, C)
                            img_array = img_array * std + mean  # Denormalize
                            img_array = np.clip(img_array, 0, 1)  # Clip to valid range
                            
                            axes[0].imshow(img_array)
                            axes[0].set_title('Original Image')
                            axes[0].axis('off')
                            
                            # SHAP heatmap - resize to match original image size
                            from scipy.ndimage import zoom as scipy_zoom
                            if shap_img_norm.shape[0] != img_array.shape[0] or shap_img_norm.shape[1] != img_array.shape[1]:
                                zoom_h = img_array.shape[0] / shap_img_norm.shape[0]
                                zoom_w = img_array.shape[1] / shap_img_norm.shape[1]
                                shap_img_resized = scipy_zoom(shap_img_norm.astype(np.float64), (zoom_h, zoom_w), order=1)
                                shap_img_resized = np.clip(shap_img_resized, 0, 1)
                            else:
                                shap_img_resized = shap_img_norm
                            
                            # Show SHAP as heatmap overlay
                            axes[1].imshow(img_array)
                            im = axes[1].imshow(shap_img_resized, cmap='jet', alpha=0.5)
                            axes[1].set_title(f'{algo_name} SHAP Attribution')
                            axes[1].axis('off')
                            plt.colorbar(im, ax=axes[1], fraction=0.046, pad=0.04)
                            
                            fig.suptitle(f'SHAP Optimization Result - {algo_name}', fontsize=14, fontweight='bold')
                            plt.tight_layout()
                            plot_path = os.path.join(run_dir, f'shap_{algo_name}.png')
                            plt.savefig(plot_path, dpi=150, bbox_inches='tight')
                            plt.close()
                            print(f"  Saved SHAP plot: {plot_path}")
                        else:
                            print(f"  Could not generate SHAP heatmap for {algo_name} (empty or None)")
                    else:
                        print(f"  SHAP values were None for {algo_name}")
                except Exception as e:
                    import traceback
                    print(f"  Failed to save SHAP plot for {algo_name}: {e}")
                    traceback.print_exc()
            except Exception as e:
                print(f"  {algo_name} failed: {e}")

    with open(os.path.join(run_dir, 'xai_params_best.json'), 'w') as f:
        json.dump(results, f, indent=2, default=str)

    # Generate XAI algorithm comparison plot
    plot_xai_algorithm_comparison(results, run_dir)

    return results

def create_html_summary(xai_dir: str, idx_to_class: Dict, run_dir: str = None, best_hyperparams: Dict = None, final_metrics: Dict = None):
    """Create HTML summary of XAI outputs with training curves and hyperparameters."""
    html_content = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>Alzheimer's Classification - Full Summary</title>
        <style>
            body { font-family: Arial, sans-serif; margin: 20px; background-color: #f5f5f5; }
            h1 { color: #333; text-align: center; }
            h2 { color: #666; border-bottom: 2px solid #4CAF50; padding-bottom: 10px; }
            h3 { color: #888; }
            .section { background: white; margin: 20px 0; padding: 20px; border-radius: 10px; box-shadow: 0 2px 5px rgba(0,0,0,0.1); }
            .class-section { margin: 20px 0; }
            .image-grid { display: flex; flex-wrap: wrap; gap: 10px; justify-content: center; }
            .image-item { border: 1px solid #ddd; padding: 5px; background: white; border-radius: 5px; }
            .image-item img { max-width: 300px; border-radius: 3px; }
            .image-item p { text-align: center; margin: 5px 0; font-size: 12px; }
            .metrics-table { width: 100%; border-collapse: collapse; margin: 10px 0; }
            .metrics-table th, .metrics-table td { border: 1px solid #ddd; padding: 10px; text-align: left; }
            .metrics-table th { background-color: #4CAF50; color: white; }
            .metrics-table tr:nth-child(even) { background-color: #f2f2f2; }
            .plot-container { text-align: center; margin: 20px 0; }
            .plot-container img { max-width: 100%; border-radius: 5px; box-shadow: 0 2px 5px rgba(0,0,0,0.1); }
        </style>
    </head>
    <body>
        <h1>🧠 Alzheimer's MRI Classification - Summary Report</h1>
        <p style="text-align: center; color: #666;">Generated: """ + datetime.now().strftime("%Y-%m-%d %H:%M:%S") + """</p>
    """

    # Best Hyperparameters Section
    if best_hyperparams:
        html_content += """
        <div class="section">
            <h2>🎯 Best Hyperparameters (from Metaheuristic Optimization)</h2>
            <table class="metrics-table">
                <tr><th>Parameter</th><th>Value</th></tr>
        """
        param_names = {
            'lr': 'Learning Rate',
            'weight_decay': 'Weight Decay',
            'dropout': 'Dropout Rate',
            'unfreeze_epoch': 'Unfreeze Backbone Epoch',
            'batch_size': 'Batch Size',
            'augment_strength': 'Augmentation Strength'
        }
        for key, value in best_hyperparams.items():
            display_name = param_names.get(key, key)
            if isinstance(value, float):
                display_value = f"{value:.6f}" if value < 0.01 else f"{value:.4f}"
            else:
                display_value = str(value)
            html_content += f"<tr><td>{display_name}</td><td>{display_value}</td></tr>"
        html_content += """
            </table>
        </div>
        """

    # Final Model Metrics Section
    if final_metrics:
        html_content += """
        <div class="section">
            <h2>📊 Final Model Performance</h2>
            <table class="metrics-table">
                <tr><th>Metric</th><th>Value</th></tr>
        """
        metric_names = {
            'test_accuracy': 'Test Accuracy',
            'test_balanced_accuracy': 'Balanced Accuracy',
            'test_macro_f1': 'Macro F1 Score',
            'best_epoch': 'Best Epoch',
            'best_val_f1': 'Best Validation F1'
        }
        for key in ['test_accuracy', 'test_balanced_accuracy', 'test_macro_f1', 'best_epoch', 'best_val_f1']:
            if key in final_metrics:
                display_name = metric_names.get(key, key)
                value = final_metrics[key]
                if isinstance(value, float):
                    display_value = f"{value:.4f}"
                else:
                    display_value = str(value)
                html_content += f"<tr><td>{display_name}</td><td>{display_value}</td></tr>"
        html_content += """
            </table>
        </div>
        """

    # Training Curves Section
    if run_dir:
        html_content += """
        <div class="section">
            <h2>📈 Training & Optimization Curves</h2>
        """
        plots_dir = os.path.join(run_dir, 'plots')
        plots_to_include = [
            ('training_curves.png', 'Training & Validation Curves'),
            ('phase_1_convergence_curves.png', 'Phase 1 Algorithm Convergence'),
            ('phase_1_algorithm_comparison.png', 'Phase 1 Algorithm Comparison'),
            ('metaheuristic_ranking.png', 'Metaheuristic Ranking'),
            ('baseline_vs_optimized.png', 'Baseline vs Optimized Performance'),
            ('xai_algorithm_comparison.png', 'XAI Algorithm Comparison')
        ]
        plots_added = False
        for plot_file, plot_title in plots_to_include:
            plot_path = os.path.join(plots_dir, plot_file)
            if os.path.exists(plot_path):
                try:
                    # Copy plot to xai_dir for HTML access
                    dest_path = os.path.join(xai_dir, plot_file)
                    shutil.copy(plot_path, dest_path)
                    # Verify the copy was successful
                    if os.path.exists(dest_path):
                        html_content += f"""
            <div class="plot-container">
                <h3>{plot_title}</h3>
                <img src="{plot_file}" alt="{plot_title}" style="max-width: 800px;">
            </div>"""
                        plots_added = True
                except Exception as e:
                    print(f"  Warning: Could not copy {plot_file}: {e}")
        
        if not plots_added:
            html_content += "<p>No training plots available.</p>"
        html_content += """
        </div>"""

    # XAI Explanations Section
    html_content += """
        <div class="section">
            <h2>XAI Explanations (Grad-CAM & SHAP)</h2>
    """
    xai_images_found = False
    for class_idx, class_name in idx_to_class.items():
        class_dir = os.path.join(xai_dir, class_name.replace(' ', '_'))
        if not os.path.exists(class_dir):
            continue
        
        # Get list of images in this class directory
        img_files = [f for f in sorted(os.listdir(class_dir)) if f.endswith('.png')]
        if not img_files:
            continue
            
        xai_images_found = True
        html_content += f"""
        <div class="class-section">
            <h3>{class_name}</h3>
            <div class="image-grid">"""
        
        for img_file in img_files:
            img_path = os.path.join(class_dir, img_file)
            if os.path.exists(img_path) and os.path.getsize(img_path) > 0:
                # Use relative path from xai_dir to the image in subdirectory
                relative_path = f"{class_name.replace(' ', '_')}/{img_file}"
                html_content += f"""
                <div class="image-item">
                    <img src="{relative_path}" alt="{img_file}">
                    <p>{img_file}</p>
                </div>"""
        
        html_content += """
            </div>
        </div>"""
    
    if not xai_images_found:
        html_content += "<p>No XAI visualizations available.</p>"

    html_content += """
        </div>
    </body>
    </html>
    """
    with open(os.path.join(xai_dir, 'summary.html'), 'w') as f:
        f.write(html_content)
    print(f"  Saved HTML summary: {os.path.join(xai_dir, 'summary.html')}")

def generate_final_explanations(model: nn.Module, loaders: Dict[str, DataLoader], metadata: Dict, xai_params: Dict, config: Config, device: torch.device, run_dir: str, best_hyperparams: Dict = None, final_metrics: Dict = None):
    """Generate final XAI explanations for sample images."""
    print("\n" + "="*50)
    print("GENERATING FINAL EXPLANATIONS")
    print("="*50)

    xai_dir = os.path.join(run_dir, 'xai_outputs')
    os.makedirs(xai_dir, exist_ok=True)

    samples_per_class = {i: {'correct': [], 'incorrect': []} for i in range(NUM_CLASSES)}
    model.eval()
    idx_to_class = metadata.get('idx_to_class', {i: f"Class_{i}" for i in range(NUM_CLASSES)})
    test_paths = metadata.get('test_paths', [])
    test_labels = metadata.get('test_labels', [])

    val_transform = T.Compose([
        T.Resize((config.IMG_SIZE, config.IMG_SIZE)),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    with torch.no_grad():
        for idx, (path, true_label) in enumerate(zip(test_paths, test_labels)):
            if all(len(samples_per_class[c]['correct']) >= config.XAI_SAMPLES_PER_CLASS and 
                   len(samples_per_class[c]['incorrect']) >= config.XAI_SAMPLES_PER_CLASS 
                   for c in range(NUM_CLASSES)):
                break
            try:
                img = Image.open(path).convert('RGB')
                img_tensor = val_transform(img).unsqueeze(0).to(device)
                output = model(img_tensor)
                pred_label = output.argmax(dim=1).item()
                category = 'correct' if pred_label == true_label else 'incorrect'
                if len(samples_per_class[true_label][category]) < config.XAI_SAMPLES_PER_CLASS:
                    samples_per_class[true_label][category].append({
                        'path': path,
                        'image': img,
                        'tensor': img_tensor.cpu(),
                        'true_label': true_label,
                        'pred_label': pred_label
                    })
            except Exception as e:
                continue

    gradcam_params = {'smoothing_sigma': 1.0, 'n_smooth_samples': 5, 'noise_std': 0.1}
    if 'gradcam' in xai_params and xai_params['gradcam']:
        first_algo = list(xai_params['gradcam'].keys())[0]
        gradcam_params = xai_params['gradcam'][first_algo].get('params', gradcam_params)

    if gradcam_params.get('n_smooth_samples', 1) > 1:
        gradcam = SmoothGradCAM(
            model,
            smoothing_sigma=gradcam_params.get('smoothing_sigma', 1.0),
            n_samples=gradcam_params.get('n_smooth_samples', 5),
            noise_std=gradcam_params.get('noise_std', 0.1)
        )
    else:
        gradcam = GradCAM(model, smoothing_sigma=gradcam_params.get('smoothing_sigma', 1.0))

    print("\nGenerating Grad-CAM visualizations...")
    for class_idx in range(NUM_CLASSES):
        class_name = idx_to_class.get(class_idx, f"Class_{class_idx}")
        class_dir = os.path.join(xai_dir, class_name.replace(' ', '_'))
        os.makedirs(class_dir, exist_ok=True)
        for category in ['correct', 'incorrect']:
            for sample_idx, sample in enumerate(samples_per_class[class_idx][category]):
                try:
                    heatmap = gradcam.generate(sample['tensor'].to(device), sample['true_label'])
                    img_array = np.array(sample['image'].resize((config.IMG_SIZE, config.IMG_SIZE)))
                    
                    # Re-normalize heatmap to ensure full 0-1 range for visualization
                    if heatmap is not None and heatmap.max() > heatmap.min():
                        heatmap = (heatmap - heatmap.min()) / (heatmap.max() - heatmap.min())
                    
                    overlay = gradcam.generate_overlay(img_array, heatmap)
                    save_path = os.path.join(class_dir, f"{category}_{sample_idx}_gradcam.png")
                    fig, axes = plt.subplots(1, 3, figsize=(12, 4))
                    axes[0].imshow(img_array)
                    axes[0].set_title('Original')
                    axes[0].axis('off')
                    axes[1].imshow(heatmap, cmap='jet', vmin=0, vmax=1)
                    axes[1].set_title('Grad-CAM Heatmap')
                    axes[1].axis('off')
                    axes[2].imshow(overlay)
                    axes[2].set_title('Overlay')
                    axes[2].axis('off')
                    pred_name = idx_to_class.get(sample['pred_label'], f"Class_{sample['pred_label']}")
                    fig.suptitle(f"True: {class_name} | Pred: {pred_name} ({category})")

                    save_path = os.path.join(class_dir, f"{category}_{sample_idx}_gradcam.png")
                    plt.savefig(save_path, dpi=150, bbox_inches='tight')
                    plt.close()
                except Exception as e:
                    print(f"  Failed to generate Grad-CAM for {class_name} {category} {sample_idx}: {e}")

    if SHAP_AVAILABLE:
        print("\nGenerating SHAP visualizations...")
        background_images = []
        for class_idx in range(NUM_CLASSES):
            for category in ['correct', 'incorrect']:
                for sample in samples_per_class[class_idx][category][:2]:
                    background_images.append(sample['tensor'])

        if background_images:
            background = torch.cat(background_images[:min(20, len(background_images))], dim=0)
            for class_idx in range(NUM_CLASSES):
                class_name = idx_to_class.get(class_idx, f"Class_{class_idx}")
                class_dir = os.path.join(xai_dir, class_name.replace(' ', '_'))
                for category in ['correct']:
                    for sample_idx, sample in enumerate(samples_per_class[class_idx][category][:2]):
                        try:
                            shap_values = run_shap_explanation(
                                model, sample['tensor'], background, device,
                                nsamples=config.SHAP_NSAMPLES
                            )
                            if shap_values is not None:
                                try:
                                    # Convert to numpy array and get shape
                                    shap_arr = np.array(shap_values)
                                    print(f"  SHAP shape for {class_name}: {shap_arr.shape}")
                                    
                                    # Reduce to (C, H, W) by taking first element of leading dimensions
                                    while len(shap_arr.shape) > 3:
                                        shap_arr = shap_arr[0]
                                    
                                    # Now get 2D heatmap by summing absolute values across channels
                                    if len(shap_arr.shape) == 3:
                                        shap_img = np.abs(shap_arr).sum(axis=0)
                                    elif len(shap_arr.shape) == 2:
                                        shap_img = np.abs(shap_arr)
                                    else:
                                        shap_img = None
                                    
                                    if shap_img is not None and shap_img.size > 0:
                                        # Normalize for visualization
                                        shap_min, shap_max = shap_img.min(), shap_img.max()
                                        if shap_max > shap_min:
                                            shap_img = (shap_img - shap_min) / (shap_max - shap_min)
                                        
                                        fig, axes = plt.subplots(1, 2, figsize=(8, 4))
                                        img_array = np.array(sample['image'].resize((config.IMG_SIZE, config.IMG_SIZE)))
                                        axes[0].imshow(img_array)
                                        axes[0].set_title('Original')
                                        axes[0].axis('off')
                                        
                                        # Resize SHAP to match image size
                                        from scipy.ndimage import zoom as scipy_zoom
                                        if shap_img.shape[0] != img_array.shape[0] or shap_img.shape[1] != img_array.shape[1]:
                                            zoom_h = img_array.shape[0] / shap_img.shape[0]
                                            zoom_w = img_array.shape[1] / shap_img.shape[1]
                                            shap_img = scipy_zoom(shap_img.astype(np.float64), (zoom_h, zoom_w), order=1)
                                            shap_img = np.clip(shap_img, 0, 1)
                                        
                                        # Show as overlay
                                        axes[1].imshow(img_array)
                                        im = axes[1].imshow(shap_img, cmap='jet', alpha=0.65, vmin=0, vmax=1)
                                        axes[1].set_title('SHAP Attribution')
                                        axes[1].axis('off')
                                        plt.colorbar(im, ax=axes[1], fraction=0.046, pad=0.04)
                                        
                                        pred_name = idx_to_class.get(sample['pred_label'], f"Class_{sample['pred_label']}")
                                        fig.suptitle(f"True: {class_name} | Pred: {pred_name}")

                                        save_path = os.path.join(class_dir, f"{category}_{sample_idx}_shap.png")
                                        plt.savefig(save_path, dpi=150, bbox_inches='tight')
                                        plt.close()
                                    else:
                                        print(f"  SHAP heatmap empty for {class_name} {category} {sample_idx}")

                                except Exception as e:
                                    print(f"  Failed SHAP for {class_name} {category} {sample_idx}: {e}")
                        except Exception as e:
                            print(f"  Failed SHAP for {class_name} {category} {sample_idx}: {e}")

    print(f"\nExplanations saved to: {xai_dir}")
    create_html_summary(xai_dir, idx_to_class, run_dir=run_dir, best_hyperparams=best_hyperparams, final_metrics=final_metrics)

def run_final_training(datasets: Dict, metadata: Dict, best_hyperparams: Dict, feature_mask: Optional[np.ndarray], config: Config, device: torch.device, run_dir: str) -> Tuple[nn.Module, Dict, Dict]:
    """Run final training with best hyperparameters."""
    print("\n" + "="*50)
    print("BASELINE MODEL TRAINING")
    print("="*50)

    # Create baseline model with default hyperparameters
    baseline_config = Config(TEST_RUN=config.TEST_RUN)
    baseline_config.EPOCHS = config.EPOCHS if not config.TEST_RUN else 1
    baseline_config.USE_WEIGHTED_SAMPLER = False
    baseline_config.USE_CLASS_WEIGHTS = False

    baseline_model = create_model(baseline_config, device)
    if feature_mask is not None:
        mask_tensor = torch.from_numpy(feature_mask).float()
        baseline_model.set_feature_mask(mask_tensor)

    baseline_loaders = create_dataloaders(datasets, metadata, baseline_config)
    baseline_model, baseline_metrics, baseline_history = train_model(
        baseline_model, baseline_loaders, baseline_config, metadata, device, run_dir,
        epochs=baseline_config.EPOCHS, verbose=False
    )

    print("\nBaseline Results:")
    print(f"  Accuracy: {baseline_metrics['test_accuracy']:.4f}")
    print(f"  Balanced Accuracy: {baseline_metrics['test_balanced_accuracy']:.4f}")
    print(f"  Macro F1: {baseline_metrics['test_macro_f1']:.4f}")

    print("\n" + "="*50)
    print("FINAL MODEL TRAINING")
    print("="*50)
    print(f"Best hyperparameters: {best_hyperparams}")

    final_config = Config(TEST_RUN=config.TEST_RUN)
    final_config.LR = best_hyperparams['lr']
    final_config.WEIGHT_DECAY = best_hyperparams['weight_decay']
    final_config.DROPOUT = best_hyperparams['dropout']
    final_config.FREEZE_BACKBONE_EPOCHS = best_hyperparams['unfreeze_epoch']
    final_config.BATCH_SIZE = best_hyperparams['batch_size']
    final_config.AUGMENT_STRENGTH = best_hyperparams['augment_strength']
    final_config.EPOCHS = config.EPOCHS + 2 if not config.TEST_RUN else 2
    final_config.EARLY_STOPPING_PATIENCE = config.EARLY_STOPPING_PATIENCE
    final_config.USE_WEIGHTED_SAMPLER = True
    final_config.USE_CLASS_WEIGHTS = True

    model = create_model(final_config, device)
    if feature_mask is not None:
        mask_tensor = torch.from_numpy(feature_mask).float()
        model.set_feature_mask(mask_tensor)
        print(f"Applied feature mask: {int(feature_mask.sum())}/{len(feature_mask)} features")

    loaders = create_dataloaders(datasets, metadata, final_config)
    model, final_metrics, history = train_model(
        model, loaders, final_config, metadata, device, run_dir,
        epochs=final_config.EPOCHS, verbose=True
    )

    final_checkpoint_path = os.path.join(run_dir, 'checkpoints', 'final_model.pt')
    torch.save({
        'model_state_dict': model.state_dict(),
        'config': final_config.to_dict(),
        'hyperparams': best_hyperparams,
        'metrics': final_metrics
    }, final_checkpoint_path)

    with open(os.path.join(run_dir, 'final_metrics.json'), 'w') as f:
        json.dump(final_metrics, f, indent=2, default=str)

    print(f"\nFinal Test Results:")
    print(f"  Accuracy: {final_metrics['test_accuracy']:.4f}")
    print(f"  Balanced Accuracy: {final_metrics['test_balanced_accuracy']:.4f}")
    print(f"  Macro F1: {final_metrics['test_macro_f1']:.4f}")

    # Compare with baseline
    print(f"\n" + "="*50)
    print("PERFORMANCE COMPARISON")
    print("="*50)
    print(f"{'Metric':<20} {'Baseline':<10} {'Optimized':<10} {'Improvement':<12}")
    print("-" * 54)
    metrics_to_compare = ['test_accuracy', 'test_balanced_accuracy', 'test_macro_f1']
    for metric in metrics_to_compare:
        baseline_val = baseline_metrics[metric]
        final_val = final_metrics[metric]
        improvement = final_val - baseline_val
        print(f"{metric.replace('test_', '').replace('_', ' ').title():<20} {baseline_val:<10.4f} {final_val:<10.4f} {improvement:<+12.4f}")

    print(f"\nOptimization Summary:")
    print(f"  - Feature Selection: {int(feature_mask.sum()) if feature_mask is not None else 'N/A'}/{len(feature_mask) if feature_mask is not None else 'N/A'} features retained")
    print(f"  - Hyperparameter Optimization: {'✓ Completed' if best_hyperparams else '✗ Failed'}")

    # Generate comparison plots
    print("\nGenerating comparison plots...")
    plot_baseline_vs_optimized(baseline_metrics, final_metrics, run_dir)
    
    # Plot confusion matrices if available
    if 'test_confusion_matrix' in baseline_metrics and 'test_confusion_matrix' in final_metrics:
        plot_confusion_matrix_comparison(
            baseline_metrics['test_confusion_matrix'],
            final_metrics['test_confusion_matrix'],
            CLASS_NAMES,
            run_dir
        )

    return model, final_metrics, baseline_metrics

print("XAI optimization and final training functions defined.")

XAI optimization and final training functions defined.


---
## 🚀 EXECUTION CELLS

Run the cells below to execute the full pipeline. Each cell represents a stage of the pipeline.

In [14]:
# Cell 13: Load Dataset

print("\n" + "#"*60)
print("# DATA LOADING")
print("#"*60)

# Create run directory
run_dir = create_run_directory(config.OUTPUT_DIR)
print(f"Run directory: {run_dir}")

# Save config
config_path = os.path.join(run_dir, 'config.json')
config.save(config_path)
print(f"Config saved to: {config_path}")

# Load dataset directly from Kaggle input path
datasets, metadata, transforms_dict = load_dataset(config, config.DATA_DIR)
loaders = create_dataloaders(datasets, metadata, config)

print(f"\n✓ Dataset loaded successfully!")
print(f"  Classes: {metadata['class_to_idx']}")
print(f"  Train samples: {len(datasets['train'])}")
print(f"  Val samples: {len(datasets['val'])}")
print(f"  Test samples: {len(datasets['test'])}")


############################################################
# DATA LOADING
############################################################
Run directory: /kaggle/working/runs/20251220_210600_93d7bdae
Config saved to: /kaggle/working/runs/20251220_210600_93d7bdae/config.json

LOADING DATASET
Using data directory: /kaggle/input/imagesoasis/Data
Found subdirectories: ['Mild Dementia', 'Moderate Dementia', 'Non Demented', 'Very mild Dementia']
Class mapping: {'Mild Dementia': 0, 'Moderate Dementia': 1, 'Non Demented': 2, 'Very mild Dementia': 3}

Total images found: 86437

Class distribution:
  Mild Dementia: 5002
  Moderate Dementia: 488
  Non Demented: 67222
  Very mild Dementia: 13725

Applying balanced subset (target: 10,000 images)
  Mild Dementia: 1800 images (target: 1800, available: 5002)
  Moderate Dementia: 488 images (target: 488, available: 488)
  Non Demented: 2912 images (target: 2912, available: 67222)
  Very mild Dementia: 2000 images (target: 2000, available: 13725)

Balanc

In [15]:
# Cell 14: ACO Feature Selection

print("\n" + "#"*60)
print("# ACO FEATURE SELECTION")
print("#"*60)

# Create initial model for feature extraction
model = create_model(config, device)

# Quick initial training for feature extraction
print("\nInitial model training for feature extraction...")
model, _, _ = train_model(
    model, loaders, config, metadata, device, run_dir,
    epochs=max(1, config.EPOCHS // 3), verbose=True
)

# Run ACO feature selection
feature_mask, aco_results = run_aco_feature_selection(model, loaders, config, device)

# Save ACO results
aco_results_path = os.path.join(run_dir, 'aco_results.json')
with open(aco_results_path, 'w') as f:
    json.dump({
        'selected_features': int(np.sum(feature_mask)) if feature_mask is not None else None,
        'total_features': len(feature_mask) if feature_mask is not None else None,
        'performance': {k: v for k, v in aco_results.items() if k != 'history'}
    }, f, indent=2)

print(f"\n✓ ACO results saved to: {aco_results_path}")

# Memory cleanup
del model
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()


############################################################
# ACO FEATURE SELECTION
############################################################


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]


Model created:
  Total parameters: 4,336,512
  Trainable parameters: 4,336,512

Initial model training for feature extraction...
Backbone frozen for first 2 epochs

  Starting Epoch 1/1...
    First batch loaded, starting training...
    Batch 10/79, Loss: 1.0562
    Batch 20/79, Loss: 1.0553
    Batch 30/79, Loss: 0.9083
    Batch 40/79, Loss: 0.7093
    Batch 50/79, Loss: 0.7585
    Batch 60/79, Loss: 0.7763
    Batch 70/79, Loss: 0.7958
    Batch 79/79, Loss: 1.0004
Epoch 1/1 (31.2s) - Train Loss: 0.9119, Val F1: 0.4076, Val Acc: 0.4472

ACO FEATURE SELECTION
Extracting features...
Feature dimension: 1280
Baseline (all features): Macro F1 = 0.5355

Running ACO Feature Selection...
  Iteration 1/2: Best Score = 0.5680, Mean = 0.5373
  Iteration 2/2: Best Score = 0.5680, Mean = 0.5340

ACO Result: Selected 640/1280 features
Best Macro F1: 0.5680
Improvement: +0.0324

✓ ACO results saved to: /kaggle/working/runs/20251220_210600_93d7bdae/aco_results.json


12595

In [16]:
# Cell 15: Phase 1 - Hyperparameter Optimization (DE, GWO, PSO, BAT)

print("\n" + "#"*60)
print("# PHASE 1: HYPERPARAMETER OPTIMIZATION")
print("#"*60)

phase1_results, convergence_histories = run_phase1_optimization(datasets, metadata, config, device, run_dir)

# Save Phase 1 results
phase1_path = os.path.join(run_dir, 'phase1_results.csv')
phase1_results.to_csv(phase1_path, index=False)
print(f"\nPhase 1 results saved to: {phase1_path}")

# Memory cleanup after Phase 1
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

# Get best hyperparams from Phase 1
best_hyperparams = get_best_hyperparams(phase1_results)
print(f"Best hyperparameters found: {best_hyperparams}")


############################################################
# PHASE 1: HYPERPARAMETER OPTIMIZATION
############################################################

PHASE 1: HYPERPARAMETER OPTIMIZATION

--- Running DE ---
    DE Eval 1: LR=0.000085, Dropout=0.44, BS=8
    First batch loaded, starting training...
    Batch 10/630, Loss: 1.2718
    Batch 20/630, Loss: 1.3984
    Batch 30/630, Loss: 1.3059
    Batch 40/630, Loss: 1.1577
    Batch 50/630, Loss: 1.3563
    Batch 60/630, Loss: 1.2008
    Batch 70/630, Loss: 1.1986
    Batch 80/630, Loss: 1.2498
    Batch 90/630, Loss: 0.9191
    Batch 100/630, Loss: 1.3772
    Batch 110/630, Loss: 0.9039
    Batch 120/630, Loss: 1.0459
    Batch 130/630, Loss: 0.9509
    Batch 140/630, Loss: 1.1276
    Batch 150/630, Loss: 0.8436
    Batch 160/630, Loss: 2.0030
    Batch 170/630, Loss: 0.7737
    Batch 180/630, Loss: 0.9905
    Batch 190/630, Loss: 1.2814
    Batch 200/630, Loss: 1.3231
    Batch 210/630, Loss: 1.1803
    Batch 220/630, Loss: 

In [17]:
# Cell 16: Phase 2 - SA Tuning of WOA and GWO

print("\n" + "#"*60)
print("# PHASE 2: SA-TUNED WOA AND GWO")
print("#"*60)

phase2_results = run_phase2_optimization(datasets, metadata, config, device, run_dir)

# Save Phase 2 results
phase2_path = os.path.join(run_dir, 'phase2_algo_param_tuning.csv')
phase2_df = pd.DataFrame([
    {'algorithm': 'WOA', 'tuned_params': str(phase2_results['WOA']['tuned_params']),
     'score': phase2_results['WOA']['best_score'], 'time': phase2_results['WOA']['time_seconds']},
    {'algorithm': 'GWO', 'tuned_params': str(phase2_results['GWO']['tuned_params']),
     'score': phase2_results['GWO']['best_score'], 'time': phase2_results['GWO']['time_seconds']}
])
phase2_df.to_csv(phase2_path, index=False)
print(f"\nPhase 2 results saved to: {phase2_path}")

# Memory cleanup after Phase 2
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

# Select top metaheuristics
print("\n" + "#"*60)
print("# SELECTING TOP METAHEURISTICS")
print("#"*60)

top_algorithms = select_top_metaheuristics(phase1_results, phase2_results, run_dir)
print(f"Top algorithms selected: {top_algorithms}")


############################################################
# PHASE 2: SA-TUNED WOA AND GWO
############################################################

PHASE 2: SA TUNING OF WOA AND GWO

--- Tuning WOA with SA ---
    First batch loaded, starting training...
    Batch 10/315, Loss: 1.3712
    Batch 20/315, Loss: 1.3467
    Batch 30/315, Loss: 1.2776
    Batch 40/315, Loss: 1.2592
    Batch 50/315, Loss: 1.3184
    Batch 60/315, Loss: 1.2773
    Batch 70/315, Loss: 1.4306
    Batch 80/315, Loss: 1.2390
    Batch 90/315, Loss: 1.2177
    Batch 100/315, Loss: 1.0245
    Batch 110/315, Loss: 1.0844
    Batch 120/315, Loss: 1.0588
    Batch 130/315, Loss: 1.2054
    Batch 140/315, Loss: 1.0581
    Batch 150/315, Loss: 1.0767
    Batch 160/315, Loss: 1.2906
    Batch 170/315, Loss: 1.0227
    Batch 180/315, Loss: 1.3057
    Batch 190/315, Loss: 1.3797
    Batch 200/315, Loss: 1.1868
    Batch 210/315, Loss: 0.8738
    Batch 220/315, Loss: 1.0163
    Batch 230/315, Loss: 0.9431
    Batch 

In [18]:
# Cell 17: XAI Optimization

print("\n" + "#"*60)
print("# XAI OPTIMIZATION")
print("#"*60)

# Create and train model for XAI
model = create_model(config, device)

# Quick training in full pipeline
train_epochs = max(1, config.EPOCHS // 2)

print(f"\nTraining model for XAI ({train_epochs} epochs)...")
model, _, _ = train_model(
    model, loaders, config, metadata, device, run_dir,
    epochs=train_epochs, verbose=True
)

# Run XAI optimization
xai_params = run_xai_optimization(
    model, loaders, top_algorithms, config, device, run_dir
)

# Save XAI params
xai_params_path = os.path.join(run_dir, 'xai_params_best.json')
with open(xai_params_path, 'w') as f:
    json.dump(xai_params, f, indent=2, default=str)
print(f"XAI parameters saved to: {xai_params_path}")

# Memory cleanup
del model
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()


############################################################
# XAI OPTIMIZATION
############################################################

Model created:
  Total parameters: 4,336,512
  Trainable parameters: 4,336,512

Training model for XAI (1 epochs)...
Backbone frozen for first 2 epochs

  Starting Epoch 1/1...
    First batch loaded, starting training...
    Batch 10/79, Loss: 1.1506
    Batch 20/79, Loss: 1.0956
    Batch 30/79, Loss: 0.9073
    Batch 40/79, Loss: 0.8330
    Batch 50/79, Loss: 0.8684
    Batch 60/79, Loss: 0.8236
    Batch 70/79, Loss: 0.8638
    Batch 79/79, Loss: 0.5658
Epoch 1/1 (18.8s) - Train Loss: 0.9007, Val F1: 0.4467, Val Acc: 0.5102

XAI OPTIMIZATION

--- Optimizing Grad-CAM parameters ---

Using GWO for Grad-CAM optimization...
  GradCAM target layer: backbone.conv_head
  GradCAM target layer: backbone.conv_head
  GradCAM target layer: backbone.conv_head
  GradCAM target layer: backbone.conv_head
  GradCAM target layer: backbone.conv_head
  GradCAM 

84599

In [19]:
# Cell 18: Final Training and Explanations

print("\n" + "#"*60)
print("# FINAL TRAINING AND EXPLANATIONS")
print("#"*60)

# Final training
final_model, final_metrics, baseline_metrics = run_final_training(
    datasets, metadata, best_hyperparams, feature_mask,
    config, device, run_dir
)

# Generate explanations
print("\nGenerating final explanations...")
generate_final_explanations(
    final_model, loaders, metadata, xai_params,
    config, device, run_dir,
    best_hyperparams=best_hyperparams,
    final_metrics=final_metrics
)


############################################################
# FINAL TRAINING AND EXPLANATIONS
############################################################

BASELINE MODEL TRAINING

Model created:
  Total parameters: 4,336,512
  Trainable parameters: 4,336,512
    First batch loaded, starting training...
    Batch 10/79, Loss: 1.2267
    Batch 20/79, Loss: 1.1121
    Batch 30/79, Loss: 1.0144
    Batch 40/79, Loss: 1.1390
    Batch 50/79, Loss: 1.1736
    Batch 60/79, Loss: 1.0358
    Batch 70/79, Loss: 1.0604
    Batch 79/79, Loss: 1.1244
    First batch loaded, starting training...
    Batch 10/79, Loss: 0.8822
    Batch 20/79, Loss: 0.9497
    Batch 30/79, Loss: 1.0031
    Batch 40/79, Loss: 0.9639
    Batch 50/79, Loss: 0.9882
    Batch 60/79, Loss: 1.0153
    Batch 70/79, Loss: 1.0835
    Batch 79/79, Loss: 1.0930

Baseline Results:
  Accuracy: 0.5917
  Balanced Accuracy: 0.5349
  Macro F1: 0.5430

FINAL MODEL TRAINING
Best hyperparameters: {'lr': np.float64(0.0003484193447750949

In [20]:
# Cell 19: Final Summary

print("\n" + "="*60)
print("PIPELINE COMPLETED SUCCESSFULLY")
print("="*60)
print(f"\nResults saved to: {run_dir}")
print(f"\nOutput files:")

outputs = [
    "config.json",
]
if feature_mask is not None:
    outputs.append("aco_results.json")
if phase1_results is not None:
    outputs.append("phase1_results.csv")
if phase2_results:
    outputs.append("phase2_algo_param_tuning.csv")
if phase2_results and phase1_results is not None:
    outputs.append("metaheuristic_ranking.csv")
outputs.append("xai_params_best.json")
outputs += [
    "checkpoints/best.pt",
    "checkpoints/final_model.pt",
    "final_metrics.json",
    "logs/",
    "plots/",
    "xai_outputs/"
]

for out in outputs:
    print(f"  - {out}")

print("="*60)


PIPELINE COMPLETED SUCCESSFULLY

Results saved to: /kaggle/working/runs/20251220_210600_93d7bdae

Output files:
  - config.json
  - aco_results.json
  - phase1_results.csv
  - phase2_algo_param_tuning.csv
  - metaheuristic_ranking.csv
  - xai_params_best.json
  - checkpoints/best.pt
  - checkpoints/final_model.pt
  - final_metrics.json
  - logs/
  - plots/
  - xai_outputs/
